In [1]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np
import matplotlib.pylab as plt
import matplotlib
%matplotlib qt
from numpy import linalg as la
from scipy.optimize import fsolve
from scipy import linalg as scpla
# import seaborn as sb
from mpl_toolkits.axes_grid1 import make_axes_locatable
from cmath import *
from mpl_toolkits.mplot3d import Axes3D
from scipy.optimize import fsolve,leastsq 
from math import tanh,cosh
import math
import time
# from sympy import *
from scipy.linalg import schur, eigvals
extras_require = {'PLOT':['matplotlib>=1.1.1,<3.0']},

In [2]:
def generate_localstatsmat_eig_desym(Nparams,Grand,rho):
    '''
    Input
      Grand 4, gee,gei,gie,gii
      Nparams: number of E and I neurons (same)
    '''
    NE,NI=Nparams[0],Nparams[1]
    N=NE+NI
    xaxiss,yaxiss =np.arange(NE*2),np.arange(NE*2)
    Xax,Yax=np.meshgrid(xaxiss,yaxiss)
    idxwhere = np.where(Xax>Yax)
    xee,xei,xie,xii=Grand[0],Grand[1],Grand[2],Grand[3]
    X,Xsym,Xasym=np.zeros((N,N)),np.zeros((N,N)),np.zeros((N,N))
    X[:NE,:NE],X[NE:,NE:]=np.random.randn(NE,NE),np.random.randn(NE,NE)
    X[:NE,NE:],X[NE:,:NE]=np.random.randn(NE,NE),np.random.randn(NE,NE)
    XT = X.copy().T

    Xsym = np.sqrt(rho)*X.copy()
    Xsym[idxwhere]=np.sqrt(rho)*XT[idxwhere]
    
    Xasym=np.sqrt(1-rho)*np.random.randn(NE*2,NE*2)
    X=Xsym.copy()+Xasym.copy()
    # components
    Xsym[:NE,:NE],Xsym[NE:,NE:]=xee*Xsym[:NE,:NE]/np.sqrt(NE),xii*Xsym[NE:,NE:]/np.sqrt(NI)
    Xsym[:NE,NE:],Xsym[NE:,:NE]=xei*Xsym[:NE,NE:]/np.sqrt(NE),xie*Xsym[NE:,:NE]/np.sqrt(NI)
    Xasym[:NE,:NE],Xasym[NE:,NE:]=xee*Xasym[:NE,:NE]/np.sqrt(NE),xii*Xasym[NE:,NE:]/np.sqrt(NI)
    Xasym[:NE,NE:],Xasym[NE:,:NE]=xei*Xasym[:NE,NE:]/np.sqrt(NE),xie*Xasym[NE:,:NE]/np.sqrt(NI)
    X=Xsym.copy()+Xasym.copy()
    # theoretical M (grandom) matrix
    gmat = np.array([[xee**2,xei**2],[xie**2,xii**2]])
    # first do not multiply at
    gaverage=0
    for i in range(2):
        for j in range(2):
            gaverage+=gmat[i,j]/2 # ntype=2
    gaverage=np.sqrt(gaverage)
    # for i in range(2):
    #     gmat[:,i]*=at[i]
    eigvgm,eigvecgm=la.eig(gmat)
    
    # properties
    Gamp=np.zeros((2,2))
    Gamp[0,0],Gamp[0,1],Gamp[1,0],Gamp[1,1]=xee,xei,xie,xii
    return (X,Xsym,Xasym,Gamp,eigvgm,eigvecgm,gaverage)

def generate_localstatsmat_eig(Nparams,Grand):
    '''
    Input 
      Grand 4, gee,gei,gie,gii
      Nparams: number of E and I neurons (same)
    '''
    xee,xei,xie,xii=Grand[0],Grand[1],Grand[2],Grand[3]
    NE,NI=Nparams[0],Nparams[1]
    N=NE+NI
    at = Nt/N   
    X=np.zeros((N,N))
    X[:NE,:NE],X[NE:,NE:]=np.random.randn(NE,NE),np.random.randn(NE,NE)
    X[:NE,NE:],X[NE:,:NE]=np.random.randn(NE,NE),np.random.randn(NE,NE)
    X[:NE,:NE],X[NE:,NE:]=xee*X[:NE,:NE]/np.sqrt(NE),xii*X[NE:,NE:]/np.sqrt(NI)
    X[:NE,NE:],X[NE:,:NE]=xei*X[:NE,NE:]/np.sqrt(NE),xie*X[NE:,:NE]/np.sqrt(NI)
    # theoretical M (grandom) matrix 
    gmat = np.array([[xee**2,xei**2],[xie**2,xii**2]])
    # first do not multiply at
    gaverage=0
    for i in range(2):
        for j in range(2):
            gaverage+=gmat[i,j]/2 # ntype=2
    gaverage=np.sqrt(gaverage)
    eigvgm,eigvecgm=la.eig(gmat)
    # properties
    Gamp=np.zeros((2,2))
    Gamp[0,0],Gamp[0,1],Gamp[1,0],Gamp[1,1]=xee,xei,xie,xii

    return (X,Gamp,eigvgm,eigvecgm,gaverage)

def generate_meanmat_eig(Nparams,JEE,JIE,JEI,JII):
    # mean value 
    # first use rank-1 structure
    NE,NI=Nparams[0],Nparams[1]
    N=NE+NI
    at = Nt/N   
    Am=np.zeros((N,N))
    Am[:NE,:NE],Am[:NE,NE:]=JEE/NE,-JEI/NI
    Am[NE:,:NE],Am[NE:,NE:]=JIE/NE,-JII/NI
    # ua,svam,vha=la.svd(Am)
    # va=vha.T
    # svam=svam[0]
    # ua,va=np.zeros((N,N)),np.zeros((N,N))
    # ua[:,0]=1.0/np.sqrt(N)
    # va[:NE,0]=JE/np.sqrt(NE)
    # va[NE:,0]=-JI/np.sqrt(NI)
    # va[:,0]/=-np.linalg.norm(va[:,0])
    # svam=np.sqrt(2*(JE**2+JI**2))
    # vha=va.T

    Jsv=np.zeros((2,2))
    Jsv[0,0],Jsv[0,1],Jsv[1,0],Jsv[1,1]=JEE,JEI,JIE,JII

    return (Am,Jsv)#,ua,svam,va)

# generate gmat, same gavg
def gmatamplitude_eig(gavgfix,typenum):
  Amplit = gavgfix*typenum
  numsample = typenum**2
  Amplitg= np.zeros(numsample)
  idxc=0
  while (1):
    if idxc>=numsample:
      Amplitg[numsample-1]=1.0-np.sum(Amplitg[:numsample-1])
      break
    p=np.random.random(1)
    Amplitg[idxc]=np.minimum(p,1.0-np.sum(Amplitg))
    if np.sum(Amplitg)>1.0:
      continue 
    elif np.sum(Amplitg)==1.0:
      break
    else:
      idxc +=1
      # Amplitg[idxc]=np.min(p,1.0-np.sum(Amplitg))
  # Amplitg=0
  Amplitg*=Amplit
  Amplitg=np.sqrt(Amplitg)
  return Amplitg


In [11]:
Nt=np.array([1000,1000])
NE,NI=Nt[0],Nt[1]
Nparams=np.array([NE,NI])
rho=0.9
JE,JI,a,b=1.0,3.5,-0.6,0.6 # already recording -3.5
a,b=0.0,0.0
JEE,JIE,JEI,JII=JE+a,JE-a,JI-b,JI+b
# generate mean Am
Am,Jsv=generate_meanmat_eig(Nparams,JEE,JIE,JEI,JII)#,ua,svam,va
Asym=Am.copy()
Asym[:NE,:NE],Asym[:NE,NE:],Asym[NE:,:NE],Asym[NE:,NE:]=rho*JE/NE,-rho*JI/NE,-rho*JI/NE,-rho*JI/NE
Aasym=Am.copy()-Asym.copy()
eigvalAasym,_=la.eig(Aasym)
eigvalAsym,_=la.eig(Asym)
eigvalA,_=la.eig(Am)
print(Aasym[1::NE,1::NE])
print((1-rho)*JE/NE,(JE+rho*JI)/NE)
print(-(1-rho)*JI/NE,-(1-rho)*JI/NE)

[[ 0.0002 -0.0007]
 [ 0.0038 -0.0007]]
0.00019999999999999996 0.0038000000000000004
-0.0006999999999999999 -0.0006999999999999999


In [17]:
grand = np.zeros(4)
gaverage=0.6
# # max r_g
# grand[0]=gaverage*np.sqrt(2) # E dominant
grand[3]=gaverage*np.sqrt(2) # I dominant
# # totally average
# grand = gaverage*np.ones(4)/np.sqrt(2.0)
# # three copy
# grand = gaverage*np.ones(4)*np.sqrt(2.0/3.0)
# grand[0]=0
X,Xsym,Xasym,Gamp,eigvgmT,eigvecgm,gaverage=generate_localstatsmat_eig_desym(Nparams,grand,rho)
Xsymd,Xasymd,Xd=Xsym.copy(),Xasym.copy(),X.copy()
for i in range(NE*2):
    Xasymd[i,i],Xd[i,i]=0,0
    Xsymd[i,i]=0.0
print(np.sum(np.sum(Xsym.T*Xsym))/2/NE)
print(np.sum(np.sum(Xasym.T*Xasym))/2/NE)
print(np.sum(np.sum(X.T*X))/2/NE)


0.2882300575373627
3.2494717365835934e-05
0.2879484747301734


In [18]:
Xtotal=Am+X.copy()
eigvalsxt,eigvecxt=la.eig(Xtotal)
Xsymtotal=Asym.copy()+Xsym.copy()
eigvalsxst,eigvecxst=la.eig(Xsymtotal)
Xasymtotal=Aasym.copy()+Xasym.copy()
eigvalsxast,eigvecxast=la.eig(Xasymtotal)

fig,ax=plt.subplots(1,3,figsize=(12,3))
ax[0].scatter(np.real(eigvalsxt),np.imag(eigvalsxt),s=5.0,c='blue',alpha=0.5)
ax[0].plot(np.sqrt(eigvgmT[1])*np.ones(20)*(1+rho),np.linspace(-1.0,1.0,20),linewidth=1.5,color='r')
ax[0].plot(np.linspace(-1.0,1.0,20),np.sqrt(eigvgmT[1])*np.ones(20)*(1-rho),linewidth=1.5,color='r')
n, bins, patches = ax[1].hist((eigvalsxst), 100, density=True, alpha=0.5)
# ax[1].plot(2*np.sqrt(rho)*np.ones(20),np.linspace(-1.0,1.0,20),linewidth=1.5,color='r')

# ax[1].hist(np.real(eigvalsxst),np.imag(eigvalsxst))
ax[2].scatter(np.real(eigvalsxast),np.imag(eigvalsxast),s=5.0,c='blue',alpha=0.5)
# ax[2].set_aspect('equal')
# eigvXsym,_=la.eig(Xsym)
# n, bins, patches = ax[3].hist(eigvXsym, 100, density=True, alpha=0.5)


In [22]:
# # svd Xsym
# _,svXsym,_=la.svd(Xsym.copy())
# # eigenspectrum Xsym
# eigvXsym,_=la.eig(Xsym.copy())
# print('compare singular values and eigenvalues:')
# print(svXsym[:2],np.sqrt(svXsym[:2]))
# print(eigvXsym[:2])

# # svd Xasym
# _,svXasym,_=la.svd(Xasym.copy())
# # eigenspectrum Xasym
# eigvXasym,_=la.eig(Xasym.copy())
# print('compare singular values and eigenvalues:')
# print(svXasym[:2],np.sqrt(svXasym[:2]))
# print(eigvXasym[:2])

# # svd Xsym
# _,svXsymtotal,_=la.svd(Xsymtotal.copy())
# print('compare singular values and eigenvalues:')
# print(svXsymtotal[:2])
# print(eigvalsxst[:2])
# # print(np.std(X[:NE,:NE]),np.std(X[:NE,NE:]),np.std(X[NE:,:NE]),np.std(X[NE:,NE:]))

## compare eigenvalues of $\bar{\mathbf{J}}^a$ and eigenvalues of $\mathbf{J}^a$
eigvmeanJa,_=la.eig(Aasym.copy())
print('compare eigenvalues of two asymmetric matrix:')
print(eigvalsxast[:2])
print(eigvmeanJa[:2])

## compare eigenvalues of $\bar{\mathbf{J}}^s$ and eigenvalues of $\mathbf{J}^s$
eigvmeanJs,_=la.eig(Asym.copy())
print('compare eigenvalues of two asymmetric matrix:')
print(eigvalsxst[:2])
print(eigvmeanJs[:2])

compare eigenvalues of two asymmetric matrix:
[-0.2504105+1.56507962j -0.2504105-1.56507962j]
[-0.25+1.56764154j -0.25-1.56764154j]
compare eigenvalues of two asymmetric matrix:
[-4.40684676+0.j  2.40222836+0.j]
[-4.3286634+0.j  2.3286634+0.j]


In [ ]:
fig,ax=plt.subplots(2,1,figsize=(3,12))
centerSym=np.zeros(NE*2)
for i in range(NE*2):
    center
n, bins, patches = ax[0].hist(eigvalAasym, 100, density=True, alpha=0.5)

# generate symmetric random matrix 01/2021

In [35]:
gaverageseries = np.linspace(0.05,1.0,20) # a series of average random gain 
symetaseries = np.linspace(0.0,1.0,11) # a series of symmetricity value
print('g:',gaverageseries[3],gaverageseries[12],gaverageseries[18])
print('eta',symetaseries[2],symetaseries[5],symetaseries[8])

g: 0.2 0.65 0.95
eta 0.2 0.5 0.8


In [35]:
# generate mean matrix
nrank=1
Nt=np.array([1000,1000])
NE,NI=Nt[0],Nt[1]
Nparams=np.array([NE,NI])
JE,JI,a,b=1.0,3.5,-0.6,0.6 # already recording -3.5
a,b=0.0,0.0
JEE,JIE,JEI,JII=JE+a,JE-a,JI-b,JI+b
# generate mean Am
Am,Jsv=generate_meanmat_eig(Nparams,JEE,JIE,JEI,JII)
# calculate the original \bar{m} and \bar{n}
eigvAm,eigveclAm,eigvecrAm=scpla.eig(Am,left=True,right=True)
meigvecAm=eigvecrAm.copy()
neigvecAm=eigveclAm.copy()
for i in range(nrank):
    normalizecoe=(np.sum(neigvecAm[:,i]*meigvecAm[:,i]))
    neigvecAm[:,i]*=(eigvAm[i]/normalizecoe)
print('normalization m & n')
print(la.norm(meigvecAm[:,0]),la.norm(neigvecAm[:,0]))
print('eigenvalues:',eigvAm[:nrank])

'''
noise correlation of m and n change with random gain $\bar{g}$ and also
symmetricity $\eta$
'''
# random gain and symmetricity
gaverageseries = np.linspace(0.05,1.0,20) # a series of average random gain 
symetaseries = np.linspace(0.0,1.0,11) # a series of symmetricity value
ngavg,neta=len(gaverageseries),len(symetaseries)
nrank,ntrial=1,20
eigvalseries,mvecseries,nvecseries=np.zeros((ngavg,neta,ntrial,nrank)),np.zeros((ngavg,neta,ntrial,NE*2,nrank)),np.zeros((ngavg,neta,ntrial,NE*2,nrank))
Reigvecseries,Leigvecseries=np.zeros((ngavg,neta,ntrial,nrank)),np.zeros((ngavg,neta,ntrial,nrank))
# generate symmetric random
for igavg, gaverage in enumerate(gaverageseries):
    for jeta,eta in enumerate(symetaseries):
        asqr=(1+np.sqrt(1-eta**2))/2.0
        aamp=np.sqrt(asqr)
        for ktrial in range(ntrial):
            Xinit=np.random.randn(NE*2,NE*2)/np.sqrt(NE)
            X0=Xinit.copy()*gaverage/np.sqrt(2)
            X=aamp*X0.T+np.sqrt(1-aamp**2)*X0
            symmetricityX=np.sum(np.sum(X.T*X))/(NE*2)
            # print(symmetricityX)
            # overall
            J = X.copy()+Am.copy()
            # properties
            eigvJ,eigrvecJ=la.eig(J)
            inveigrvecJ=la.inv(eigrvecJ)
            meig = np.squeeze(eigrvecJ[:,:nrank].copy())
            meig = np.reshape(meig,(NE*2,nrank))
            neig = np.squeeze(inveigrvecJ[:nrank,:].copy()) # inverse
            neig = np.reshape(neig.T,(NE*2,nrank))
            # record Reigvec,Leigvec
            Reigvecseries[igavg,jeta,ktrial,:,i]=meig[:,i].copy()
            Leigvecseries[igavg,jeta,ktrial,:,i]=neig[:,i].copy()
            for i in range(nrank):
                neig[:,i]*=eigvJ[i]
                eigvalseries[igavg,jeta,ktrial,i]=eigvJ[i]
                mvecseries[igavg,jeta,ktrial,:,i]=meig[:,i].copy()
                nvecseries[igavg,jeta,ktrial,:,i]=neig[:,i].copy()

            #     print('rank-',i)
            #     print(la.norm(meig[:,i]),la.norm(neig[:,i]))
            # fig,ax=plt.subplots(figsize=(6,6))
            # ax.scatter(meig[:NE],neig[:NE],s=5.0,c='red',alpha=0.25)
            # ax.scatter(meig[NE:],neig[NE:],s=5.0,c='blue',alpha=0.25)
            # ax.set_aspect('equal')

            # recording 
            


KeyboardInterrupt: 

In [6]:
# saving data
now = time.strftime("%Y-%m-%d-%H_%M_%S",time.localtime(time.time()))
Jparameters = np.zeros(4+1)
Jparameters[0],Jparameters[1],Jparameters[2],Jparameters[3]=JEE,JEI,JIE,JII
Jparameters[4]=NE

# gaverageseries = np.linspace(0.05,1.0,20) # a series of average random gain 
# symetaseries = np.linspace(0.0,1.0,3) # a series of symmetricity value
# ngavg,neta=len(gaverageseries),len(symetaseries)
# nrank,ntrial=1,5
# eigvalseries,mvecseries,nvecseries

np.savez(now+'_symmetricity_data',Jparameters=Jparameters,gaverageseries=gaverageseries,symetaseries=symetaseries,mvecseries=mvecseries,nvecseries=nvecseries,eigvalseries=eigvalseries,Reigvecseries=Reigvecseries,Leigvecseries=Leigvecseries,meigvecAm=meigvecAm,neigvecAm=neigvecAm,eigvAm=eigvAm)

## testing Gaussian distribution; For single gaverage and eta; scatter plot

In [24]:
# generate mean matrix
nrank=1
Nt=np.array([1000,1000])
NE,NI=Nt[0],Nt[1]
Nparams=np.array([NE,NI])
JE,JI,a,b=1.0,3.5,-0.6,0.6 # already recording -3.5
a,b=0.0,0.0
JEE,JIE,JEI,JII=JE+a,JE-a,JI-b,JI+b
# generate mean Am
Am,Jsv=generate_meanmat_eig(Nparams,JEE,JIE,JEI,JII)
# calculate the original \bar{m} and \bar{n}
eigvAm,eigveclAm,eigvecrAm=scpla.eig(Am,left=True,right=True)
meigvecAm=eigvecrAm.copy()
neigvecAm=eigveclAm.copy()
for i in range(nrank):
    normalizecoe=(np.sum(neigvecAm[:,i]*meigvecAm[:,i]))
    neigvecAm[:,i]*=(eigvAm[i]/normalizecoe)
print('normalization m & n')
print(la.norm(meigvecAm[:,0]),la.norm(neigvecAm[:,0]))
print('eigenvalues:',eigvAm[:nrank])

'''
noise correlation of m and n change with random gain $\bar{g}$ and also
symmetricity $\eta$
'''
eta = 0.6
asqr=(1+np.sqrt(1-eta**2))/2.0
aamp=np.sqrt(asqr)
gaverage=1.0
Xinit=np.random.randn(NE*2,NE*2)/np.sqrt(NE)
X0=Xinit.copy()*gaverage/np.sqrt(2)
X=aamp*X0.T+np.sqrt(1-aamp**2)*X0
x=np.arange(NE*2)
XX,YY=np.meshgrid(x,x)
idx = np.where(XX<YY)
testvalue = X0[idx]

fig,ax=plt.subplots(figsize=(9,9))
n, bins, patches = ax.hist(np.real(testvalue), 20, density=True, alpha=0.5)


normalization m & n
1.0000000000000004 5.147815070493496
eigenvalues: [-2.5+0.j]


In [28]:
testvalue = X*X.T
sigsym = np.mean(np.reshape(testvalue,-1))
fig,ax=plt.subplots(1,2,figsize=(12,9))
n, bins, patches = ax[0].hist(np.reshape(np.real(testvalue),((NE*2)**2,1)), 100, density=True, alpha=0.5)
testvalue = X0*X0.T
sigAsym = np.mean(np.reshape(testvalue,-1))
# fig,ax=plt.subplots(1,2,figsize=(12,9))
n, bins, patches = ax[1].hist(np.reshape(np.real(testvalue),((NE*2)**2,1)), 100, density=True, alpha=0.5)

## read in data and test how local statistics change with $\bar{g}$ and $\eta$; change with a series --- multiple

In [128]:
strfile ='2021-01-29-05_51_27_symmetricity_data'
data = np.load(strfile+'.npz')#'2021-01-19-12_02_53_heterogeneousgII_cloudsdata.npz')
mvecseries,nvecseries=data['mvecseries'],data['nvecseries']
gaverageseries=data['gaverageseries']
eigvAm=data['eigvAm']
''' how eigenvalue outlier changes with g_bar '''
# three eta 0.2,0.5,0.8
# np.zeros((ngavg,neta,ntrial,nrank))
ngavg,neta,ntrial,N,nrank=np.shape(mvecseries)
gaverageseries = np.linspace(0.05,1.0,20) # a series of average random gain 
symetaseries = np.linspace(0.0,1.0,11) # a series of symmetricity value
idxg=10
idxeta=np.array([3,6,9])#([2,5,8])
# idxeta1=np.where(symetaseries==0.2)
# idxeta2=np.where(symetaseries==0.5)
# idxeta3=np.where(symetaseries==0.8)
''' get eigenvalue outlier series  '''
eigval_outliers_mean,eigval_outliers_std=np.zeros((neta,ngavg)),np.zeros((neta,ngavg))
eigval_outliers_eta = np.zeros(neta)
idxgavg=10
for i in range(neta):
    eigval_outliers=np.sum(np.squeeze(mvecseries[:,i,:,:,0])*np.squeeze(nvecseries[:,i,:,:,0]),axis=2)
    # print('shape:',np.shape(eigval_outliers))
    eigval_outliers_mean[i,:]=np.mean(eigval_outliers,axis=1)# ngavg,neta,ntrial
    eigval_outliers_eta[i]=np.mean(eigval_outliers_mean[i,:])#eigval_outliers_mean[i,idxgavg]#
    eigval_outliers_std[i,:]=np.std(eigval_outliers,axis=0)
# print('choose eta:',idxeta1,',',idxeta2,',',idxeta3)




In [129]:
fig,ax=plt.subplots(figsize=(6,4))
xticks=np.linspace(0,1.0,3)
xlims=[-0.01,1.06]
yticks=np.linspace(-3.0,-2.5,3)
ylims=[-3.01,-2.49]

# rank-1
clset='rgb'
for i in range(neta):
    ax.plot(gaverageseries,eigval_outliers_mean[i,:], c=(1-symetaseries[i])*np.ones(3))#,label=r'$\eta=$'+str(symetaseries[i]))
ax.plot(gaverageseries,eigvAm[0]*np.ones(ngavg),c='red',linestyle='--',linewidth=2.0)
ax.set_xlim(xlims)
ax.set_xticks(xticks)
ax.set_ylim(ylims)
ax.set_yticks(yticks)
''' how eigenvalue outlier changes with eta '''
fig,ax=plt.subplots(figsize=(6,4))
for i in range(neta):
    ax.scatter(symetaseries[i],eigval_outliers_eta[i], c=(1-symetaseries[i])*np.ones(3),cmap='viridis')
ax.plot(gaverageseries,eigvAm[0]*np.ones(ngavg),c='red',linestyle='--',linewidth=2.0)
ax.set_xlim(xlims)
ax.set_xticks(xticks)
ax.set_ylim(ylims)
ax.set_yticks(yticks)
# ax.set_aspect('equal')

*c* argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with *x* & *y*.  Please use the *color* keyword-argument or provide a 2-D array with a single row if you intend to specify the same RGB or RGBA value for all points.
*c* argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with *x* & *y*.  Please use the *color* keyword-argument or provide a 2-D array with a single row if you intend to specify the same RGB or RGBA value for all points.
*c* argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with *x* & *y*.  Please use the *color* keyword-argument or provide a 2-D array with a single row if you intend to specify the same RGB or RGBA value for all points.
*c* argument looks like a single numeric RGB o

## noise correlation

In [3]:
from matplotlib.patches import Ellipse
import matplotlib.transforms as transforms
def confidence_ellipse(x, y, ax, n_std=3.0, facecolor='none', **kwargs):
    """
    Create a plot of the covariance confidence ellipse of `x` and `y`

    Parameters
    ----------
    x, y : array_like, shape (n, )
        Input data.

    ax : matplotlib.axes.Axes
        The axes object to draw the ellipse into.

    n_std : float
        The number of standard deviations to determine the ellipse's radiuses.

    Returns
    -------
    matplotlib.patches.Ellipse

    Other parameters
    ----------------
    kwargs : `~matplotlib.patches.Patch` properties
    """
    if x.size != y.size:
        raise ValueError("x and y must be the same size")

    cov = np.cov(x, y)
    print(np.shape(cov))
    pearson = cov[0, 1]/np.sqrt(cov[0, 0] * cov[1, 1])
    # Using a special case to obtain the eigenvalues of this
    # two-dimensionl dataset.
    ell_radius_x = np.sqrt(1 + pearson)
    ell_radius_y = np.sqrt(1 - pearson)
    ellipse = Ellipse((0, 0),
        width=ell_radius_x * 2,
        height=ell_radius_y * 2,
        facecolor=facecolor,
        **kwargs)

    # Calculating the stdandard deviation of x from
    # the squareroot of the variance and multiplying
    # with the given number of standard deviations.
    scale_x = np.sqrt(cov[0, 0]) * n_std
    mean_x = np.mean(x)

    # calculating the stdandard deviation of y ...
    scale_y = np.sqrt(cov[1, 1]) * n_std
    mean_y = np.mean(y)

    transf = transforms.Affine2D() \
        .rotate_deg(45) \
        .scale(scale_x, scale_y) \
        .translate(mean_x, mean_y)

    ellipse.set_transform(transf + ax.transData)
    return ax.add_patch(ellipse)

In [207]:
idxeta,idxtrial=8,8#2#5#8
idxgavg=18#18#12#3#np.array([1,10,19])
meigvec=np.reshape(np.squeeze(mvecseries[idxgavg,idxeta,idxtrial,:,0]),(2*NE,1))
neigvec=np.reshape(np.squeeze(nvecseries[idxgavg,idxeta,idxtrial,:,0]),(2*NE,1))
## validate the normalization
lambdarecon=neigvec.T@meigvec
print('eigenvalue outlier:',lambdarecon)

# fig,ax=plt.subplots(3,3,figsize=(8,8))
# mE,nE=meigvec[:NE,0].copy(),neigvec[:NE,0].copy()
# confidence_ellipse(np.real(nE),np.real(mE),ax[0][0],edgecolor='orange')
# ax[0][0].scatter(nE,mE,s=10,c='red',alpha=0.5)
# mI,nI=meigvec[NE:,0].copy(),neigvec[NE:,0].copy()
# confidence_ellipse(np.real(nI),np.real(mI),ax[0][0],edgecolor='cyan')
# ax[0][0].scatter(nI,mI,s=10,c='blue',alpha=0.5)

eigenvalue outlier: [[-2.82684491]]


In [208]:
idxeta,idxtrial=8,8#2#5#8
idxgavg=18#18#12#3#np.array([1,10,19])
meigvec=np.reshape(np.squeeze(mvecseries[idxgavg,idxeta,idxtrial,:,0]),(2*NE,1))
neigvec=np.reshape(np.squeeze(nvecseries[idxgavg,idxeta,idxtrial,:,0]),(2*NE,1))
## validate the normalization
lambdarecon=neigvec.T@meigvec
print('eigenvalue outlier:',lambdarecon)

# fig,ax=plt.subplots(3,3,figsize=(8,8))
# mE,nE=meigvec[:NE,0].copy(),neigvec[:NE,0].copy()
# confidence_ellipse(np.real(nE),np.real(mE),ax[0][0],edgecolor='orange')
# ax[0][0].scatter(nE,mE,s=10,c='red',alpha=0.5)
# mI,nI=meigvec[NE:,0].copy(),neigvec[NE:,0].copy()
# confidence_ellipse(np.real(nI),np.real(mI),ax[0][0],edgecolor='cyan')
# ax[0][0].scatter(nI,mI,s=10,c='blue',alpha=0.5)
meanmE,meanmI,meannE,meannI=np.zeros(nrank),np.zeros(nrank),np.zeros(nrank),np.zeros(nrank)
meigvecloading,neigvecloading=meigvec.copy(),neigvec.copy()
scale_std=3.0
for irank in range(nrank):
    # m vector
    # puring E
    mEmat = np.squeeze(meigvecloading[:NE,irank])
    meanmEtotal,stdmEtotal = np.mean(mEmat),np.std(mEmat)
    varmE = mEmat - meanmEtotal
    idxwhere = np.where(np.abs(varmE)>scale_std*stdmEtotal)
    mEmat[idxwhere]=meanmEtotal
    meigvecloading[:NE,irank]=mEmat
    meanmE[irank]=np.mean(mEmat)

    # puring I
    mImat = np.squeeze(meigvecloading[NE:,irank])
    meanmItotal,stdmItotal = np.mean(mImat),np.std(mImat)
    varmI = mImat - meanmItotal
    idxwhere = np.where(np.abs(varmI)>scale_std*stdmItotal)
    mImat[idxwhere]=meanmItotal
    meigvecloading[NE:,irank]=mImat
    meanmI[irank]=np.mean(mImat)
    
    # n vector
    # puring E
    nEmat = np.squeeze(neigvecloading[:NE,irank])
    meannEtotal,stdnEtotal = np.mean(nEmat),np.std(nEmat)
    varnE = nEmat - meannEtotal
    idxwhere = np.where(np.abs(varnE)>scale_std*stdnEtotal)
    nEmat[idxwhere]=meannEtotal
    neigvecloading[:NE,irank]=nEmat
    meannE[irank]=np.mean(nEmat)

    # puring I
    nImat = np.squeeze(neigvecloading[NE:,irank])
    meannItotal,stdnItotal = np.mean(nImat),np.std(nImat)
    varnI = nImat - meannItotal
    idxwhere = np.where(np.abs(varnI)>scale_std*stdnItotal)
    nImat[idxwhere]=meannItotal
    neigvecloading[NE:,irank]=nImat
    meannI[irank]=np.mean(nImat)

''' calculate directions '''
noiseE,noiseI = np.zeros((NE,2,nrank)),np.zeros((NE,2,nrank))
dirvecE,dirvecI=np.zeros((2,2)),np.zeros((2,2))
for irank in range(nrank):
    # E 0 M 1 N
    noiseE[:,0,irank],noiseE[:,1,irank]= np.squeeze(neigvecloading[:NE,irank])-meannE[irank],np.squeeze(meigvecloading[:NE,irank])-meanmE[irank]
    noiseI[:,0,irank],noiseI[:,1,irank]= np.squeeze(neigvecloading[NE:,irank])-meannI[irank],np.squeeze(meigvecloading[NE:,irank])-meanmI[irank]
# m1n1
covdirE,covdirI=np.squeeze(noiseE[:,:,0]).T@np.squeeze(noiseE[:,:,0]),np.squeeze(noiseI[:,:,0]).T@np.squeeze(noiseI[:,:,0])
_,dirvecE=la.eig(covdirE)
_,dirvecI=la.eig(covdirI)

xlims =[-0.31,0.31]
xticks = np.linspace(-0.3,0.3,3)
ylims =[-0.11,0.11]
yticks = np.linspace(-0.1,0.1,3)
# figure for eigenvectors
fig,ax=plt.subplots(figsize=(12,3))
#
ii,jj=2,2


ax.scatter(neigvecloading[NE:,0],meigvecloading[NE:,0],s=5.0,c='blue',alpha=0.5)
ax.scatter(neigvecloading[:NE,0],meigvecloading[:NE,0],s=5.0,c='red',alpha=0.5)

ax.plot([meannE[0],meannE[0]+dirvecE[0,0]],[meanmE[0],meanmE[0]+dirvecE[1,0]],color='r',linestyle='--',linewidth=1.5)
ax.plot([meannE[0],meannE[0]+dirvecE[0,1]],[meanmE[0],meanmE[0]+dirvecE[1,1]],color='r',linestyle='--',linewidth=1.5)
ax.plot([meannI[0],meannI[0]+dirvecI[0,0]],[meanmI[0],meanmI[0]+dirvecI[1,0]],color='b',linestyle='--',linewidth=1.5)
ax.plot([meannI[0],meannI[0]+dirvecI[0,1]],[meanmI[0],meanmI[0]+dirvecI[1,1]],color='b',linestyle='--',linewidth=1.5)

mE,nE=meigvecloading[:NE,0].copy(),neigvecloading[:NE,0].copy()
confidence_ellipse(np.real(nE),np.real(mE),ax,edgecolor='orange')
ax.scatter(nE,mE,s=10,c='red',alpha=0.5)
mI,nI=meigvecloading[NE:,0].copy(),neigvecloading[NE:,0].copy()
confidence_ellipse(np.real(nI),np.real(mI),ax,edgecolor='cyan')
ax.scatter(nI,mI,s=10,c='blue',alpha=0.5)
ax.axvline(c='grey', lw=1)
ax.axhline(c='grey', lw=1)


ax.set_xlim(xlims)
ax.set_ylim(ylims)
ax.set_xticks(xticks)
ax.set_yticks(yticks)
ax.set_aspect('equal')

# ax[ii][jj].scatter(neigvecloading[NE:,0],meigvecloading[NE:,0],s=5.0,c='blue',alpha=0.5)
# ax[ii][jj].scatter(neigvecloading[:NE,0],meigvecloading[:NE,0],s=5.0,c='red',alpha=0.5)

# ax[ii][jj].plot([meannE[0],meannE[0]+dirvecE[0,0]],[meanmE[0],meanmE[0]+dirvecE[1,0]],color='r',linestyle='--',linewidth=1.5)
# ax[ii][jj].plot([meannE[0],meannE[0]+dirvecE[0,1]],[meanmE[0],meanmE[0]+dirvecE[1,1]],color='r',linestyle='--',linewidth=1.5)
# ax[ii][jj].plot([meannI[0],meannI[0]+dirvecI[0,0]],[meanmI[0],meanmI[0]+dirvecI[1,0]],color='b',linestyle='--',linewidth=1.5)
# ax[ii][jj].plot([meannI[0],meannI[0]+dirvecI[0,1]],[meanmI[0],meanmI[0]+dirvecI[1,1]],color='b',linestyle='--',linewidth=1.5)

# mE,nE=meigvecloading[:NE,0].copy(),neigvecloading[:NE,0].copy()
# confidence_ellipse(np.real(nE),np.real(mE),ax[ii][jj],edgecolor='orange')
# ax[ii][jj].scatter(nE,mE,s=10,c='red',alpha=0.5)
# mI,nI=meigvecloading[NE:,0].copy(),neigvecloading[NE:,0].copy()
# confidence_ellipse(np.real(nI),np.real(mI),ax[ii][jj],edgecolor='cyan')
# ax[ii][jj].scatter(nI,mI,s=10,c='blue',alpha=0.5)
# ax[ii][jj].axvline(c='grey', lw=1)
# ax[ii][jj].axhline(c='grey', lw=1)


# ax[ii][jj].set_xlim(xlims)
# ax[ii][jj].set_ylim(ylims)
# ax[ii][jj].set_xticks(xticks)
# ax[ii][jj].set_yticks(yticks)
# ax[ii][jj].set_aspect('equal')

(2, 2)
(2, 2)


## heterogeneous connectivity

In [36]:
# generate mean matrix
nrank=1
Nt=np.array([1000,1000])
NE,NI=Nt[0],Nt[1]
Nparams=np.array([NE,NI])
JE,JI,a,b=1.0,3.5,-0.6,0.6 # already recording -3.5
a,b=0.0,0.0
JEE,JIE,JEI,JII=JE+a,JE-a,JI-b,JI+b
# generate mean Am
Am,Jsv=generate_meanmat_eig(Nparams,JEE,JIE,JEI,JII)
# calculate the original \bar{m} and \bar{n}
eigvAm,eigveclAm,eigvecrAm=scpla.eig(Am,left=True,right=True)
meigvecAm=eigvecrAm.copy()
neigvecAm=eigveclAm.copy()
for i in range(nrank):
    normalizecoe=(np.sum(neigvecAm[:,i]*meigvecAm[:,i]))
    neigvecAm[:,i]*=(eigvAm[i]/normalizecoe)
print('normalization m & n')
print(la.norm(meigvecAm[:,0]),la.norm(neigvecAm[:,0]))
print('eigenvalues:',eigvAm[:nrank])

'''
noise correlation of m and n change with random gain $\bar{g}$ and also
symmetricity $\eta$
'''
# random gain and symmetricity
gaverageseries = np.linspace(0.1,1.0,10) # a series of average random gain 
symetaseries = np.linspace(0.1,1.0,10) # a series of symmetricity value
ngavg,neta=len(gaverageseries),len(symetaseries)
nrank,ntrial=1,20
eigvalseries,mvecseries,nvecseries=np.zeros((ngavg,neta,ntrial,nrank)),np.zeros((ngavg,neta,ntrial,NE*2,nrank)),np.zeros((ngavg,neta,ntrial,NE*2,nrank))
Reigvecseries,Leigvecseries=np.zeros((ngavg,neta,ntrial,NE*2,nrank)),np.zeros((ngavg,neta,ntrial,NE*2,nrank))
# generate symmetric random
for igavg, gaverage in enumerate(gaverageseries):
    for jeta,eta in enumerate(symetaseries):
        asqr=(1+np.sqrt(1-eta**2))/2.0
        aamp=np.sqrt(asqr)
        for ktrial in range(ntrial):
            Xinit=np.random.randn(NE*2,NE*2)/np.sqrt(NE)
            X0=Xinit.copy()
            X=aamp*X0.T+np.sqrt(1-aamp**2)*X0
            # # ''' homogeneous '''
            # X*=(gaverage/np.sqrt(2))
            # # ''' heterogeneous rII '''
            # X[:NE,:NE]=0
            # X[:NE,NE:]=0
            # X[NE:,:NE]=0
            # X[NE:,NE:]*=np.sqrt(2)*gaverage
            # ''' heterogeneous not rEE '''
            X[:NE,:NE]=0
            X[:NE,NE:]*=(np.sqrt(2.0/3.0)*gaverage)
            X[NE:,:NE]*=(np.sqrt(2.0/3.0)*gaverage)
            X[NE:,NE:]*=(np.sqrt(2.0/3.0)*gaverage)

            symmetricityX=np.sum(np.sum(X.T*X))/(NE*2)
            print('g_bar and eta')
            print(gaverage,';',eta)
            print('symmetricity factor')
            print(symmetricityX)
            # overall
            J = X.copy()+Am.copy()
            # properties
            eigvJ,eigrvecJ=la.eig(J)
            inveigrvecJ=la.inv(eigrvecJ)
            meig = np.squeeze(eigrvecJ[:,:nrank].copy())
            meig = np.reshape(meig,(NE*2,nrank))
            neig = np.squeeze(inveigrvecJ[:nrank,:].copy()) # inverse
            neig = np.reshape(neig.T,(NE*2,nrank))
            # record Reigvec,Leigvec
            Reigvecseries[igavg,jeta,ktrial,:,0]=meig[:,0].copy()
            Leigvecseries[igavg,jeta,ktrial,:,0]=neig[:,0].copy()
            for i in range(nrank):
                neig[:,i]*=eigvJ[i]
                eigvalseries[igavg,jeta,ktrial,i]=eigvJ[i]
                mvecseries[igavg,jeta,ktrial,:,i]=meig[:,i].copy()
                nvecseries[igavg,jeta,ktrial,:,i]=neig[:,i].copy()

            # recording 
            


normalization m & n
1.0000000000000004 5.147815070493501
eigenvalues: [-2.5+0.j]
g_bar and eta
0.1 ; 0.1
symmetricity factor
0.0010169992719743658
g_bar and eta
0.1 ; 0.1
symmetricity factor
0.00100067221380969
g_bar and eta
0.1 ; 0.1
symmetricity factor
0.0009897319423627627
g_bar and eta
0.1 ; 0.1
symmetricity factor
0.0010065581563392084
g_bar and eta
0.1 ; 0.1
symmetricity factor
0.001013832138846501
g_bar and eta
0.1 ; 0.1
symmetricity factor
0.0010055247691417323
g_bar and eta
0.1 ; 0.1
symmetricity factor
0.0010106898642545184
g_bar and eta
0.1 ; 0.1
symmetricity factor
0.0009949256492880444
g_bar and eta
0.1 ; 0.1
symmetricity factor
0.001009201314304855
g_bar and eta
0.1 ; 0.1
symmetricity factor
0.0010144353164988984
g_bar and eta
0.1 ; 0.1
symmetricity factor
0.0010053958434285966
g_bar and eta
0.1 ; 0.1
symmetricity factor
0.0010100139919589084
g_bar and eta
0.1 ; 0.1
symmetricity factor
0.000995534232177208
g_bar and eta
0.1 ; 0.1
symmetricity factor
0.001004822099172793
g

KeyboardInterrupt: 

In [21]:
# saving data
now = time.strftime("%Y-%m-%d-%H_%M_%S",time.localtime(time.time()))
Jparameters = np.zeros(4+1)
Jparameters[0],Jparameters[1],Jparameters[2],Jparameters[3]=JEE,JEI,JIE,JII
Jparameters[4]=NE

# gaverageseries = np.linspace(0.05,1.0,20) # a series of average random gain 
# symetaseries = np.linspace(0.0,1.0,3) # a series of symmetricity value
# ngavg,neta=len(gaverageseries),len(symetaseries)
# nrank,ntrial=1,5
# eigvalseries,mvecseries,nvecseries

np.savez(now+'_heterNotrEEsymmetricity_data',Jparameters=Jparameters,gaverageseries=gaverageseries,symetaseries=symetaseries,mvecseries=mvecseries,nvecseries=nvecseries,eigvalseries=eigvalseries,Reigvecseries=Reigvecseries,Leigvecseries=Leigvecseries,meigvecAm=meigvecAm,neigvecAm=neigvecAm,eigvAm=eigvAm)

In [11]:
# generate mean matrix
nrank=1
Nt=np.array([1000,1000])
NE,NI=Nt[0],Nt[1]
Nparams=np.array([NE,NI])
JE,JI,a,b=1.0,1.5,-0.6,0.6 # already recording -3.5
a,b=0.0,0.0
JEE,JIE,JEI,JII=JE+a,JE-a,JI-b,JI+b
# generate mean Am
Am,Jsv=generate_meanmat_eig(Nparams,JEE,JIE,JEI,JII)
# calculate the original \bar{m} and \bar{n}
eigvAm,eigveclAm,eigvecrAm=scpla.eig(Am,left=True,right=True)
meigvecAm=eigvecrAm.copy()
neigvecAm=eigveclAm.copy()
for i in range(nrank):
    normalizecoe=(np.sum(neigvecAm[:,i]*meigvecAm[:,i]))
    neigvecAm[:,i]*=(eigvAm[i]/normalizecoe)
print('normalization m & n')
print(la.norm(meigvecAm[:,0]),la.norm(neigvecAm[:,0]))
print('eigenvalues:',eigvAm[:nrank])

'''
noise correlation of m and n change with random gain $\bar{g}$ and also
symmetricity $\eta$
'''
# random gain and symmetricity
gaverageseries = np.linspace(0.0,1.0,3) # a series of average random gain 
symetaseries = np.linspace(0.0,1.0,3) # a series of symmetricity value
ngavg,neta=len(gaverageseries),len(symetaseries)
nrank,ntrial=1,20
eigvalseries,mvecseries,nvecseries=np.zeros((ntrial,nrank)),np.zeros((ntrial,NE*2,nrank)),np.zeros((ntrial,NE*2,nrank))
Reigvecseries,Leigvecseries=np.zeros((ntrial,NE*2,nrank)),np.zeros((ntrial,NE*2,nrank))
# generate symmetric random
gaverage,eta=gaverageseries[2],symetaseries[2]
# random 
while(1):
    grand = gmatamplitude_eig(gaverage**2,2)
    idxeff = np.where(grand>0)
    if (np.shape(idxeff)[1]>3):
        np.random.shuffle(grand)
        break
print('random amp:',grand)
asqr=(1+np.sqrt(1-eta**2))/2.0
aamp=np.sqrt(asqr)
for ktrial in range(ntrial):
    Xinit=np.random.randn(NE*2,NE*2)/np.sqrt(NE)
    X0=Xinit.copy()#*gaverage#/np.sqrt(2)#1.0/np.sqrt(2)
    X=aamp*X0.T+np.sqrt(1-aamp**2)*X0
    # # ''' homogeneous '''
    # X*=(gaverage/np.sqrt(2))
    # # ''' heterogeneous rII '''
    # X[:NE,:NE]=0
    # X[:NE,NE:]=0
    # X[NE:,:NE]=0
    # X[NE:,NE:]*=np.sqrt(2)*gaverage
    # # ''' heterogeneous not rEE '''
    # X[:NE,:NE]=0
    # X[:NE,NE:]*=(np.sqrt(2.0/3.0)*gaverage)
    # X[NE:,:NE]*=(np.sqrt(2.0/3.0)*gaverage)
    # X[NE:,NE:]*=(np.sqrt(2.0/3.0)*gaverage)
    # random 
    X[:NE,:NE]*=grand[0]
    X[:NE,NE:]*=grand[1]
    X[NE:,:NE]*=grand[2]
    X[NE:,NE:]*=grand[3]
    theogrand = (grand[1]*grand[2]*2+grand[0]**2+grand[3]**2)/2
    symmetricityX=np.sum(np.sum(X.T*X))/(NE*2)
    print(symmetricityX,theogrand*eta)
    # overall
    J = X.copy()+Am.copy()
    # properties
    eigvJ,eigrvecJ=la.eig(J)
    inveigrvecJ=la.inv(eigrvecJ)
    meig = np.squeeze(eigrvecJ[:,:nrank].copy())
    meig = np.reshape(meig,(NE*2,nrank))
    neig = np.squeeze(inveigrvecJ[:nrank,:].copy()) # inverse
    neig = np.reshape(neig.T,(NE*2,nrank))
    # record Reigvec,Leigvec
    Reigvecseries[ktrial,:,0]=meig[:,0].copy()
    Leigvecseries[ktrial,:,0]=neig[:,0].copy()
    for i in range(nrank):
        neig[:,i]*=eigvJ[i]
        eigvalseries[ktrial,i]=eigvJ[i]
        mvecseries[ktrial,:,i]=meig[:,i].copy()
        nvecseries[ktrial,:,i]=neig[:,i].copy()

            


normalization m & n
1.0000000000000004 2.549509756796389
eigenvalues: [-0.5+0.j]
random amp: [0.11614064 0.95248801 0.53507819 0.89048823]
0.912891128828765 0.9128845196915535
0.9134712400743228 0.9128845196915535
0.9137027502569341 0.9128845196915535
0.912425454866241 0.9128845196915535
0.9137826447013887 0.9128845196915535
0.9133293758281411 0.9128845196915535
0.9132374668328475 0.9128845196915535
0.912496586422046 0.9128845196915535
0.9143824404808478 0.9128845196915535
0.9142538412100409 0.9128845196915535
0.9131772195878992 0.9128845196915535
0.9122944301770786 0.9128845196915535
0.9142736624580231 0.9128845196915535
0.9130025462976911 0.9128845196915535
0.9141113806917907 0.9128845196915535
0.9128125064643237 0.9128845196915535
0.9126986088000278 0.9128845196915535
0.913128178028083 0.9128845196915535
0.9124194133588184 0.9128845196915535
0.9117949597752492 0.9128845196915535


In [39]:
# saving data
now = time.strftime("%Y-%m-%d-%H_%M_%S",time.localtime(time.time()))
Jparameters = np.zeros(4+1)
Jparameters[0],Jparameters[1],Jparameters[2],Jparameters[3]=JEE,JEI,JIE,JII
Jparameters[4]=NE

gparameters=np.zeros(4+2)
gparameters[:4],gparameters[4],gparameters[5]=grand,gaverage,eta
np.savez(now+'_hetersymmetricity_data',Jparameters=Jparameters,gparameters=gparameters,mvecseries=mvecseries,nvecseries=nvecseries,eigvalseries=eigvalseries,Reigvecseries=Reigvecseries,Leigvecseries=Leigvecseries,meigvecAm=meigvecAm,neigvecAm=neigvecAm,eigvAm=eigvAm)

In [8]:
# saving data
now = time.strftime("%Y-%m-%d-%H_%M_%S",time.localtime(time.time()))
Jparameters = np.zeros(4+1)
Jparameters[0],Jparameters[1],Jparameters[2],Jparameters[3]=JEE,JEI,JIE,JII
Jparameters[4]=NE

# gaverageseries = np.linspace(0.05,1.0,20) # a series of average random gain 
# symetaseries = np.linspace(0.0,1.0,3) # a series of symmetricity value
# ngavg,neta=len(gaverageseries),len(symetaseries)
# nrank,ntrial=1,5
# eigvalseries,mvecseries,nvecseries

np.savez(now+'_hetersymmetricity_data',Jparameters=Jparameters,gaverageseries=gaverageseries,symetaseries=symetaseries,mvecseries=mvecseries,nvecseries=nvecseries,eigvalseries=eigvalseries,Reigvecseries=Reigvecseries,Leigvecseries=Leigvecseries,meigvecAm=meigvecAm,neigvecAm=neigvecAm,eigvAm=eigvAm)

[0.  0.5 1. ] [0.05 1.  ]


## the mean connectivity affects the maximum rotation?


In [10]:
# generate mean matrix
nrank=1
Nt=np.array([1000,1000])
NE,NI=Nt[0],Nt[1]
Nparams=np.array([NE,NI])
JE,JI,a,b=1.0,3.5,-0.6,0.6 # already recording -3.5
a,b=0.0,0.0
JIseries = np.linspace(2.0,5.0,20)
nrank,ntrial,nJI=1,2,len(JIseries)

eigvalseries,mvecseries,nvecseries=np.zeros((nJI,ntrial,nrank)),np.zeros((nJI,ntrial,NE*2,nrank)),np.zeros((nJI,ntrial,NE*2,nrank))
Reigvecseries,Leigvecseries=np.zeros((nJI,ntrial,NE*2,nrank)),np.zeros((nJI,ntrial,NE*2,nrank))
thetamaxseries=np.zeros((nJI,ntrial,2,2))# EI, DIR1,DIR2
# generate symmetric random
gaverage,eta=1.0,0.8
# # random 
# while(1):
#     grand = gmatamplitude_eig(gaverage**2,2)
#     idxeff = np.where(grand>0)
#     if (np.shape(idxeff)[1]>3):
#         np.random.shuffle(grand)
#         break
# print('random amp:',grand)
asqr=(1+np.sqrt(1-eta**2))/2.0
aamp=np.sqrt(asqr)
for idxJI, JI in enumerate(JIseries):
    JEE,JIE,JEI,JII=JE+a,JE-a,JI-b,JI+b
    # generate mean Am
    Am,Jsv=generate_meanmat_eig(Nparams,JEE,JIE,JEI,JII)
    # calculate the original \bar{m} and \bar{n}
    eigvAm,eigveclAm,eigvecrAm=scpla.eig(Am,left=True,right=True)
    print('original outlier:',eigvAm[0])
    meigvecAm=eigvecrAm.copy()
    neigvecAm=eigveclAm.copy()
    for i in range(nrank):
        normalizecoe=(np.sum(neigvecAm[:,i]*meigvecAm[:,i]))
        neigvecAm[:,i]*=(eigvAm[i]/normalizecoe)
    # print('normalization m & n')
    # print(la.norm(meigvecAm[:,0]),la.norm(neigvecAm[:,0]))
    # print('eigenvalues:',eigvAm[:nrank])

    '''
    noise correlation of m and n change with random gain $\bar{g}$ and also
    symmetricity $\eta$
    '''

    ''' homogeneous case '''
    if idxJI==0:
        gaverage*=np.abs((eigvAm[0]/(1+eta)*0.8))
    gmat = gaverage**2/2*np.ones((2,2))
    eigvgm,eigvecgm=la.eig(gmat)
    radius_g = np.sqrt(eigvgm[0])
    # ''' rII case '''
    # xee,xei,xie,xii=0,gaverage*np.sqrt(2/3),gaverage*np.sqrt(2/3),gaverage*np.sqrt(2/3)
    # gmat = np.array([[xee**2,xei**2],[xie**2,xii**2]])
    # eigvgm,eigvecgm=la.eig(gmat)
    # radius_g = np.sqrt(eigvgm[0])
    for ktrial in range(ntrial):
        Xinit=np.random.randn(NE*2,NE*2)/np.sqrt(NE)
        X0=Xinit.copy()#*gaverage#/np.sqrt(2)#1.0/np.sqrt(2)
        X=aamp*X0.T+np.sqrt(1-aamp**2)*X0
        # ''' homogeneous '''
        X*=(gaverage/np.sqrt(2))
        # # ''' heterogeneous rII '''
        # X[:NE,:NE]=0
        # X[:NE,NE:]=0
        # X[NE:,:NE]=0
        # X[NE:,NE:]*=np.sqrt(2)*gaverage
        # # ''' heterogeneous not rEE '''
        # X[:NE,:NE]=0
        # X[:NE,NE:]*=(np.sqrt(2.0/3.0)*gaverage)
        # X[NE:,:NE]*=(np.sqrt(2.0/3.0)*gaverage)
        # X[NE:,NE:]*=(np.sqrt(2.0/3.0)*gaverage)
        # # random 
        # X[:NE,:NE]*=grand[0]
        # X[:NE,NE:]*=grand[1]
        # X[NE:,:NE]*=grand[2]
        # X[NE:,NE:]*=grand[3]
        # theogrand = (grand[1]*grand[2]*2+grand[0]**2+grand[3]**2)/2
        # symmetricityX=np.sum(np.sum(X.T*X))/(NE*2)
        # print(symmetricityX,theogrand*eta)
        # overall
        J = X.copy()+Am.copy()
        # properties
        eigvJ,eigrvecJ=la.eig(J)
        inveigrvecJ=la.inv(eigrvecJ)
        meig = np.squeeze(eigrvecJ[:,:nrank].copy())
        meig = np.reshape(meig,(NE*2,nrank))
        neig = np.squeeze(inveigrvecJ[:nrank,:].copy()) # inverse
        neig = np.reshape(neig.T,(NE*2,nrank))
        # record Reigvec,Leigvec
        Reigvecseries[idxJI,ktrial,:,0]=meig[:,0].copy()
        Leigvecseries[idxJI,ktrial,:,0]=neig[:,0].copy()
        for i in range(nrank):
            neig[:,i]*=eigvJ[i]
            eigvalseries[idxJI,ktrial,i]=eigvJ[i]
            mvecseries[idxJI,ktrial,:,i]=meig[:,i].copy()
            nvecseries[idxJI,ktrial,:,i]=neig[:,i].copy()

        meanmE,meanmI,meannE,meannI=np.zeros(nrank),np.zeros(nrank),np.zeros(nrank),np.zeros(nrank)
        meigvecloading,neigvecloading=mvecseries[idxJI,ktrial,:,0].copy(),nvecseries[idxJI,ktrial,:,0].copy()
        meigvecloading,neigvecloading=np.reshape(meigvecloading,(2*NE,1)),np.reshape(neigvecloading,(2*NE,1))
        scale_std=3.0
        for irank in range(nrank):
            # m vector
            # puring E
            mEmat = np.squeeze(meigvecloading[:NE,irank])
            meanmEtotal,stdmEtotal = np.mean(mEmat),np.std(mEmat)
            varmE = mEmat - meanmEtotal
            idxwhere = np.where(np.abs(varmE)>scale_std*stdmEtotal)
            mEmat[idxwhere]=meanmEtotal
            meigvecloading[:NE,irank]=mEmat
            meanmE[irank]=np.mean(mEmat)

            # puring I
            mImat = np.squeeze(meigvecloading[NE:,irank])
            meanmItotal,stdmItotal = np.mean(mImat),np.std(mImat)
            varmI = mImat - meanmItotal
            idxwhere = np.where(np.abs(varmI)>scale_std*stdmItotal)
            mImat[idxwhere]=meanmItotal
            meigvecloading[NE:,irank]=mImat
            meanmI[irank]=np.mean(mImat)
            
            # n vector
            # puring E
            nEmat = np.squeeze(neigvecloading[:NE,irank])
            meannEtotal,stdnEtotal = np.mean(nEmat),np.std(nEmat)
            varnE = nEmat - meannEtotal
            idxwhere = np.where(np.abs(varnE)>scale_std*stdnEtotal)
            nEmat[idxwhere]=meannEtotal
            neigvecloading[:NE,irank]=nEmat
            meannE[irank]=np.mean(nEmat)

            # puring I
            nImat = np.squeeze(neigvecloading[NE:,irank])
            meannItotal,stdnItotal = np.mean(nImat),np.std(nImat)
            varnI = nImat - meannItotal
            idxwhere = np.where(np.abs(varnI)>scale_std*stdnItotal)
            nImat[idxwhere]=meannItotal
            neigvecloading[NE:,irank]=nImat
            meannI[irank]=np.mean(nImat)

        ''' calculate directions '''
        noiseE,noiseI = np.zeros((NE,2,nrank)),np.zeros((NE,2,nrank))
        dirvecE,dirvecI=np.zeros((2,2)),np.zeros((2,2))
        for irank in range(nrank):
            # E 0 M 1 N
            noiseE[:,0,irank],noiseE[:,1,irank]= np.squeeze(neigvecloading[:NE,irank])-meannE[irank],np.squeeze(meigvecloading[:NE,irank])-meanmE[irank]
            noiseI[:,0,irank],noiseI[:,1,irank]= np.squeeze(neigvecloading[NE:,irank])-meannI[irank],np.squeeze(meigvecloading[NE:,irank])-meanmI[irank]
        # m1n1
        covdirE,covdirI=np.squeeze(noiseE[:,:,0]).T@np.squeeze(noiseE[:,:,0]),np.squeeze(noiseI[:,:,0]).T@np.squeeze(noiseI[:,:,0])
        _,dirvecE=la.eig(covdirE)
        thetamaxseries[idxJI,ktrial,0,0],thetamaxseries[idxJI,ktrial,0,1]  =np.arctan(dirvecE[0,0]/dirvecE[1,0]),np.arctan(dirvecE[0,1]/dirvecE[1,1])
        _,dirvecI=la.eig(covdirI)  
        thetamaxseries[idxJI,ktrial,1,0],thetamaxseries[idxJI,ktrial,1,1]  =np.arctan(dirvecI[0,0]/dirvecI[1,0]),np.arctan(dirvecI[0,1]/dirvecI[1,1])
              


original outlier: (-1.0000000000000022+0j)
original outlier: (-1.1578947368421089+0j)
original outlier: (-1.3157894736841982+0j)
original outlier: (-1.4736842105263197+0j)
original outlier: (-1.63157894736842+0j)
original outlier: (-1.789473684210523+0j)
original outlier: (-1.947368421052623+0j)
original outlier: (-2.1052631578947505+0j)
original outlier: (-2.2631578947368394+0j)
original outlier: (-2.421052631578927+0j)
original outlier: (-2.578947368421047+0j)
original outlier: (-2.7368421052631624+0j)
original outlier: (-2.894736842105278+0j)
original outlier: (-3.052631578947398+0j)
original outlier: (-3.210526315789453+0j)
original outlier: (-3.3684210526315566+0j)
original outlier: (-3.5263157894736787+0j)
original outlier: (-3.6842105263157987+0j)
original outlier: (-3.842105263157919+0j)
original outlier: (-4+0j)


In [15]:
fig,ax=plt.subplots(figsize=(4,4))
# ax.scatter(np.real(eigvJ),np.imag(eigvJ),s=5.0,c='blue',alpha=0.5)
ax.plot(JIseries,np.abs(thetamaxseries[:,1,1,0]))
xlims =[-1.1,1.1]
xticks = np.linspace(-1.,1.0,3)
ylims =[-1.1,1.1]
yticks = np.linspace(-1.,1.,3)
# ax.set_xlim(xlims)
# ax.set_ylim(ylims)
# ax.set_xticks(xticks)
# ax.set_yticks(yticks)
# ax.set_aspect('equal')


## quantitatively measure the relationship between noise covariance($\theta$) with change in $\lambda$

In [17]:
# generate mean matrix
nrank=1
Nt=np.array([1000,1000])
NE,NI=Nt[0],Nt[1]
Nparams=np.array([NE,NI])
JE,JI,a,b=1.0,3.5,-0.6,0.6 # already recording -3.5
a,b=0.0,0.0
JEE,JIE,JEI,JII=JE+a,JE-a,JI-b,JI+b
# generate mean Am
Am,Jsv=generate_meanmat_eig(Nparams,JEE,JIE,JEI,JII)
# calculate the original \bar{m} and \bar{n}
eigvAm,eigveclAm,eigvecrAm=scpla.eig(Am,left=True,right=True)
print('original outlier:',eigvAm[0])
meigvecAm=eigvecrAm.copy()
neigvecAm=eigveclAm.copy()
for i in range(nrank):
    normalizecoe=(np.sum(neigvecAm[:,i]*meigvecAm[:,i]))
    neigvecAm[:,i]*=(eigvAm[i]/normalizecoe)

symetaseries = np.linspace(0.0,1.0,11)
nrank,ntrial,neta=1,20,len(symetaseries)

eigvalseries,mvecseries,nvecseries=np.zeros((neta,ntrial,nrank)),np.zeros((neta,ntrial,NE*2,nrank)),np.zeros((neta,ntrial,NE*2,nrank))
Reigvecseries,Leigvecseries=np.zeros((neta,ntrial,NE*2,nrank)),np.zeros((nJI,ntrial,NE*2,nrank))
thetamaxseries=np.zeros((neta,ntrial,2,2))# EI, DIR1,DIR2
# generate symmetric random
gaverage=1.0
# # random 
# while(1):
#     grand = gmatamplitude_eig(gaverage**2,2)
#     idxeff = np.where(grand>0)
#     if (np.shape(idxeff)[1]>3):
#         np.random.shuffle(grand)
#         break
# print('random amp:',grand)
for idxeta, eta in enumerate(symetaseries):
    '''
    noise correlation of m and n change with random gain $\bar{g}$ and also
    symmetricity $\eta$
    '''

    ''' homogeneous case '''
    gmat = gaverage**2/2*np.ones((2,2))
    eigvgm,eigvecgm=la.eig(gmat)
    radius_g = np.sqrt(eigvgm[0])
    print('radius-g:',radius_g)
    asqr=(1+np.sqrt(1-eta**2))/2.0
    aamp=np.sqrt(asqr)  
    # ''' rII case '''
    # xee,xei,xie,xii=0,gaverage*np.sqrt(2/3),gaverage*np.sqrt(2/3),gaverage*np.sqrt(2/3)
    # gmat = np.array([[xee**2,xei**2],[xie**2,xii**2]])
    # eigvgm,eigvecgm=la.eig(gmat)
    # radius_g = np.sqrt(eigvgm[0])
    for ktrial in range(ntrial):
        Xinit=np.random.randn(NE*2,NE*2)/np.sqrt(NE)
        X0=Xinit.copy()#*gaverage#/np.sqrt(2)#1.0/np.sqrt(2)
        X=aamp*X0.T+np.sqrt(1-aamp**2)*X0
        # ''' homogeneous '''
        X*=(gaverage/np.sqrt(2))
        # # ''' heterogeneous rII '''
        # X[:NE,:NE]=0
        # X[:NE,NE:]=0
        # X[NE:,:NE]=0
        # X[NE:,NE:]*=np.sqrt(2)*gaverage
        # # ''' heterogeneous not rEE '''
        # X[:NE,:NE]=0
        # X[:NE,NE:]*=(np.sqrt(2.0/3.0)*gaverage)
        # X[NE:,:NE]*=(np.sqrt(2.0/3.0)*gaverage)
        # X[NE:,NE:]*=(np.sqrt(2.0/3.0)*gaverage)
        # # random 
        # X[:NE,:NE]*=grand[0]
        # X[:NE,NE:]*=grand[1]
        # X[NE:,:NE]*=grand[2]
        # X[NE:,NE:]*=grand[3]
        # theogrand = (grand[1]*grand[2]*2+grand[0]**2+grand[3]**2)/2
        # symmetricityX=np.sum(np.sum(X.T*X))/(NE*2)
        # print(symmetricityX,theogrand*eta)
        # overall
        J = X.copy()+Am.copy()
        # properties
        eigvJ,eigrvecJ=la.eig(J)
        inveigrvecJ=la.inv(eigrvecJ)
        meig = np.squeeze(eigrvecJ[:,:nrank].copy())
        meig = np.reshape(meig,(NE*2,nrank))
        neig = np.squeeze(inveigrvecJ[:nrank,:].copy()) # inverse
        neig = np.reshape(neig.T,(NE*2,nrank))
        # record Reigvec,Leigvec
        Reigvecseries[idxeta,ktrial,:,0]=meig[:,0].copy()
        Leigvecseries[idxeta,ktrial,:,0]=neig[:,0].copy()
        for i in range(nrank):
            neig[:,i]*=eigvJ[i]
            eigvalseries[idxeta,ktrial,i]=eigvJ[i]
            mvecseries[idxeta,ktrial,:,i]=meig[:,i].copy()
            nvecseries[idxeta,ktrial,:,i]=neig[:,i].copy()

        meanmE,meanmI,meannE,meannI=np.zeros(nrank),np.zeros(nrank),np.zeros(nrank),np.zeros(nrank)
        meigvecloading,neigvecloading=mvecseries[idxeta,ktrial,:,0].copy(),nvecseries[idxeta,ktrial,:,0].copy()
        meigvecloading,neigvecloading=np.reshape(meigvecloading,(2*NE,1)),np.reshape(neigvecloading,(2*NE,1))
        scale_std=3.0
        for irank in range(nrank):
            # m vector
            # puring E
            mEmat = np.squeeze(meigvecloading[:NE,irank])
            meanmEtotal,stdmEtotal = np.mean(mEmat),np.std(mEmat)
            varmE = mEmat - meanmEtotal
            idxwhere = np.where(np.abs(varmE)>scale_std*stdmEtotal)
            mEmat[idxwhere]=meanmEtotal
            meigvecloading[:NE,irank]=mEmat
            meanmE[irank]=np.mean(mEmat)

            # puring I
            mImat = np.squeeze(meigvecloading[NE:,irank])
            meanmItotal,stdmItotal = np.mean(mImat),np.std(mImat)
            varmI = mImat - meanmItotal
            idxwhere = np.where(np.abs(varmI)>scale_std*stdmItotal)
            mImat[idxwhere]=meanmItotal
            meigvecloading[NE:,irank]=mImat
            meanmI[irank]=np.mean(mImat)
            
            # n vector
            # puring E
            nEmat = np.squeeze(neigvecloading[:NE,irank])
            meannEtotal,stdnEtotal = np.mean(nEmat),np.std(nEmat)
            varnE = nEmat - meannEtotal
            idxwhere = np.where(np.abs(varnE)>scale_std*stdnEtotal)
            nEmat[idxwhere]=meannEtotal
            neigvecloading[:NE,irank]=nEmat
            meannE[irank]=np.mean(nEmat)

            # puring I
            nImat = np.squeeze(neigvecloading[NE:,irank])
            meannItotal,stdnItotal = np.mean(nImat),np.std(nImat)
            varnI = nImat - meannItotal
            idxwhere = np.where(np.abs(varnI)>scale_std*stdnItotal)
            nImat[idxwhere]=meannItotal
            neigvecloading[NE:,irank]=nImat
            meannI[irank]=np.mean(nImat)

        ''' calculate directions '''
        noiseE,noiseI = np.zeros((NE,2,nrank)),np.zeros((NE,2,nrank))
        dirvecE,dirvecI=np.zeros((2,2)),np.zeros((2,2))
        for irank in range(nrank):
            # E 0 M 1 N
            noiseE[:,0,irank],noiseE[:,1,irank]= np.squeeze(neigvecloading[:NE,irank])-meannE[irank],np.squeeze(meigvecloading[:NE,irank])-meanmE[irank]
            noiseI[:,0,irank],noiseI[:,1,irank]= np.squeeze(neigvecloading[NE:,irank])-meannI[irank],np.squeeze(meigvecloading[NE:,irank])-meanmI[irank]
        # m1n1
        covdirE,covdirI=np.squeeze(noiseE[:,:,0]).T@np.squeeze(noiseE[:,:,0]),np.squeeze(noiseI[:,:,0]).T@np.squeeze(noiseI[:,:,0])
        _,dirvecE=la.eig(covdirE)
        thetamaxseries[idxeta,ktrial,0,0],thetamaxseries[idxeta,ktrial,0,1]  =np.arctan(dirvecE[0,0]/dirvecE[1,0]),np.arctan(dirvecE[0,1]/dirvecE[1,1])
        _,dirvecI=la.eig(covdirI)  
        thetamaxseries[idxeta,ktrial,1,0],thetamaxseries[idxeta,ktrial,1,1]  =np.arctan(dirvecI[0,0]/dirvecI[1,0]),np.arctan(dirvecI[0,1]/dirvecI[1,1])
              


original outlier: (-2.499999999999994+0j)
radius-g: 1.0
radius-g: 1.0
radius-g: 1.0
radius-g: 1.0
radius-g: 1.0
radius-g: 1.0
radius-g: 1.0
radius-g: 1.0
radius-g: 1.0
radius-g: 1.0
radius-g: 1.0


## original mean's effect on noise covariance (hardly find, no outlier)

In [ ]:
# generate mean matrix
nrank=1
Nt=np.array([1000,1000])
NE,NI=Nt[0],Nt[1]
Nparams=np.array([NE,NI])
JE,JI,a,b=1.0,1.5,-0.6,0.6 # already recording -3.5
a,b=0.0,0.0
JEE,JIE,JEI,JII=JE+a,JE-a,JI-b,JI+b
# generate mean Am
Am,Jsv=generate_meanmat_eig(Nparams,JEE,JIE,JEI,JII)
# calculate the original \bar{m} and \bar{n}
eigvAm,eigveclAm,eigvecrAm=scpla.eig(Am,left=True,right=True)
meigvecAm=eigvecrAm.copy()
neigvecAm=eigveclAm.copy()
for i in range(nrank):
    normalizecoe=(np.sum(neigvecAm[:,i]*meigvecAm[:,i]))
    neigvecAm[:,i]*=(eigvAm[i]/normalizecoe)
print('normalization m & n')
print(la.norm(meigvecAm[:,0]),la.norm(neigvecAm[:,0]))
print('eigenvalues:',eigvAm[:nrank])

'''
noise correlation of m and n change with random gain $\bar{g}$ and also
symmetricity $\eta$
'''
# random gain and symmetricity
gaverageseries = np.linspace(0.0,1.0,3) # a series of average random gain 
symetaseries = np.linspace(0.0,1.0,3) # a series of symmetricity value
ngavg,neta=len(gaverageseries),len(symetaseries)
nrank,ntrial=1,2
eigvalseries,mvecseries,nvecseries=np.zeros((ntrial,nrank)),np.zeros((ntrial,NE*2,nrank)),np.zeros((ntrial,NE*2,nrank))
Reigvecseries,Leigvecseries=np.zeros((ntrial,NE*2,nrank)),np.zeros((ntrial,NE*2,nrank))
# generate symmetric random
gaverage,eta=gaverageseries[2],symetaseries[2]
# random 
while(1):
    grand = gmatamplitude_eig(gaverage**2,2)
    idxeff = np.where(grand>0)
    if (np.shape(idxeff)[1]>3):
        np.random.shuffle(grand)
        break
print('random amp:',grand)
asqr=(1+np.sqrt(1-eta**2))/2.0
aamp=np.sqrt(asqr)
for ktrial in range(ntrial):
    Xinit=np.random.randn(NE*2,NE*2)/np.sqrt(NE)
    X0=Xinit.copy()#*gaverage#/np.sqrt(2)#1.0/np.sqrt(2)
    X=aamp*X0.T+np.sqrt(1-aamp**2)*X0
    # # ''' homogeneous '''
    # X*=(gaverage/np.sqrt(2))
    # # ''' heterogeneous rII '''
    # X[:NE,:NE]=0
    # X[:NE,NE:]=0
    # X[NE:,:NE]=0
    # X[NE:,NE:]*=np.sqrt(2)*gaverage
    # # ''' heterogeneous not rEE '''
    # X[:NE,:NE]=0
    # X[:NE,NE:]*=(np.sqrt(2.0/3.0)*gaverage)
    # X[NE:,:NE]*=(np.sqrt(2.0/3.0)*gaverage)
    # X[NE:,NE:]*=(np.sqrt(2.0/3.0)*gaverage)
    # random 
    X[:NE,:NE]*=grand[0]
    X[:NE,NE:]*=grand[1]
    X[NE:,:NE]*=grand[2]
    X[NE:,NE:]*=grand[3]
    theogrand = (grand[1]*grand[2]*2+grand[0]**2+grand[3]**2)/2
    symmetricityX=np.sum(np.sum(X.T*X))/(NE*2)
    print(symmetricityX,theogrand*eta)
    # overall
    J = X.copy()+Am.copy()
    # properties
    eigvJ,eigrvecJ=la.eig(J)
    inveigrvecJ=la.inv(eigrvecJ)
    meig = np.squeeze(eigrvecJ[:,:nrank].copy())
    meig = np.reshape(meig,(NE*2,nrank))
    neig = np.squeeze(inveigrvecJ[:nrank,:].copy()) # inverse
    neig = np.reshape(neig.T,(NE*2,nrank))
    # record Reigvec,Leigvec
    Reigvecseries[ktrial,:,0]=meig[:,0].copy()
    Leigvecseries[ktrial,:,0]=neig[:,0].copy()
    for i in range(nrank):
        neig[:,i]*=eigvJ[i]
        eigvalseries[ktrial,i]=eigvJ[i]
        mvecseries[ktrial,:,i]=meig[:,i].copy()
        nvecseries[ktrial,:,i]=neig[:,i].copy()

            


In [17]:
idxeta,idxtrial=8,3#2#5#8
idxgavg=18#18#12#3#np.array([1,10,19])
meigvec=np.reshape(np.squeeze(mvecseries[idxtrial,:,0]),(2*NE,1))
neigvec=np.reshape(np.squeeze(nvecseries[idxtrial,:,0]),(2*NE,1))
## validate the normalization
lambdarecon=neigvec.T@meigvec
print('eigenvalue outlier:',lambdarecon)

# fig,ax=plt.subplots(3,3,figsize=(8,8))
# mE,nE=meigvec[:NE,0].copy(),neigvec[:NE,0].copy()
# confidence_ellipse(np.real(nE),np.real(mE),ax[0][0],edgecolor='orange')
# ax[0][0].scatter(nE,mE,s=10,c='red',alpha=0.5)
# mI,nI=meigvec[NE:,0].copy(),neigvec[NE:,0].copy()
# confidence_ellipse(np.real(nI),np.real(mI),ax[0][0],edgecolor='cyan')
# ax[0][0].scatter(nI,mI,s=10,c='blue',alpha=0.5)
meanmE,meanmI,meannE,meannI=np.zeros(nrank),np.zeros(nrank),np.zeros(nrank),np.zeros(nrank)
meigvecloading,neigvecloading=meigvec.copy(),neigvec.copy()
scale_std=3.0
for irank in range(nrank):
    # m vector
    # puring E
    mEmat = np.squeeze(meigvecloading[:NE,irank])
    meanmEtotal,stdmEtotal = np.mean(mEmat),np.std(mEmat)
    varmE = mEmat - meanmEtotal
    idxwhere = np.where(np.abs(varmE)>scale_std*stdmEtotal)
    mEmat[idxwhere]=meanmEtotal
    meigvecloading[:NE,irank]=mEmat
    meanmE[irank]=np.mean(mEmat)

    # puring I
    mImat = np.squeeze(meigvecloading[NE:,irank])
    meanmItotal,stdmItotal = np.mean(mImat),np.std(mImat)
    varmI = mImat - meanmItotal
    idxwhere = np.where(np.abs(varmI)>scale_std*stdmItotal)
    mImat[idxwhere]=meanmItotal
    meigvecloading[NE:,irank]=mImat
    meanmI[irank]=np.mean(mImat)
    
    # n vector
    # puring E
    nEmat = np.squeeze(neigvecloading[:NE,irank])
    meannEtotal,stdnEtotal = np.mean(nEmat),np.std(nEmat)
    varnE = nEmat - meannEtotal
    idxwhere = np.where(np.abs(varnE)>scale_std*stdnEtotal)
    nEmat[idxwhere]=meannEtotal
    neigvecloading[:NE,irank]=nEmat
    meannE[irank]=np.mean(nEmat)

    # puring I
    nImat = np.squeeze(neigvecloading[NE:,irank])
    meannItotal,stdnItotal = np.mean(nImat),np.std(nImat)
    varnI = nImat - meannItotal
    idxwhere = np.where(np.abs(varnI)>scale_std*stdnItotal)
    nImat[idxwhere]=meannItotal
    neigvecloading[NE:,irank]=nImat
    meannI[irank]=np.mean(nImat)

''' calculate directions '''
noiseE,noiseI = np.zeros((NE,2,nrank)),np.zeros((NE,2,nrank))
dirvecE,dirvecI=np.zeros((2,2)),np.zeros((2,2))
for irank in range(nrank):
    # E 0 M 1 N
    noiseE[:,0,irank],noiseE[:,1,irank]= np.squeeze(neigvecloading[:NE,irank])-meannE[irank],np.squeeze(meigvecloading[:NE,irank])-meanmE[irank]
    noiseI[:,0,irank],noiseI[:,1,irank]= np.squeeze(neigvecloading[NE:,irank])-meannI[irank],np.squeeze(meigvecloading[NE:,irank])-meanmI[irank]
# m1n1
covdirE,covdirI=np.squeeze(noiseE[:,:,0]).T@np.squeeze(noiseE[:,:,0]),np.squeeze(noiseI[:,:,0]).T@np.squeeze(noiseI[:,:,0])
_,dirvecE=la.eig(covdirE)
_,dirvecI=la.eig(covdirI)

xlims =[-0.31,0.31]
xticks = np.linspace(-0.3,0.3,3)
ylims =[-0.11,0.11]
yticks = np.linspace(-0.1,0.1,3)
# figure for eigenvectors
fig,ax=plt.subplots(figsize=(12,3))
#
ii,jj=2,2


ax.scatter(neigvecloading[NE:,0],meigvecloading[NE:,0],s=5.0,c='blue',alpha=0.5)
ax.scatter(neigvecloading[:NE,0],meigvecloading[:NE,0],s=5.0,c='red',alpha=0.5)

ax.plot([meannE[0],meannE[0]+dirvecE[0,0]],[meanmE[0],meanmE[0]+dirvecE[1,0]],color='r',linestyle='--',linewidth=1.5)
ax.plot([meannE[0],meannE[0]+dirvecE[0,1]],[meanmE[0],meanmE[0]+dirvecE[1,1]],color='r',linestyle='--',linewidth=1.5)
ax.plot([meannI[0],meannI[0]+dirvecI[0,0]],[meanmI[0],meanmI[0]+dirvecI[1,0]],color='b',linestyle='--',linewidth=1.5)
ax.plot([meannI[0],meannI[0]+dirvecI[0,1]],[meanmI[0],meanmI[0]+dirvecI[1,1]],color='b',linestyle='--',linewidth=1.5)

mE,nE=meigvecloading[:NE,0].copy(),neigvecloading[:NE,0].copy()
confidence_ellipse(np.real(nE),np.real(mE),ax,edgecolor='orange')
ax.scatter(nE,mE,s=10,c='red',alpha=0.5)
mI,nI=meigvecloading[NE:,0].copy(),neigvecloading[NE:,0].copy()
confidence_ellipse(np.real(nI),np.real(mI),ax,edgecolor='cyan')
ax.scatter(nI,mI,s=10,c='blue',alpha=0.5)
ax.axvline(c='grey', lw=1)
ax.axhline(c='grey', lw=1)


ax.set_xlim(xlims)
ax.set_ylim(ylims)
ax.set_xticks(xticks)
ax.set_yticks(yticks)
ax.set_aspect('equal')

# ax[ii][jj].scatter(neigvecloading[NE:,0],meigvecloading[NE:,0],s=5.0,c='blue',alpha=0.5)
# ax[ii][jj].scatter(neigvecloading[:NE,0],meigvecloading[:NE,0],s=5.0,c='red',alpha=0.5)

# ax[ii][jj].plot([meannE[0],meannE[0]+dirvecE[0,0]],[meanmE[0],meanmE[0]+dirvecE[1,0]],color='r',linestyle='--',linewidth=1.5)
# ax[ii][jj].plot([meannE[0],meannE[0]+dirvecE[0,1]],[meanmE[0],meanmE[0]+dirvecE[1,1]],color='r',linestyle='--',linewidth=1.5)
# ax[ii][jj].plot([meannI[0],meannI[0]+dirvecI[0,0]],[meanmI[0],meanmI[0]+dirvecI[1,0]],color='b',linestyle='--',linewidth=1.5)
# ax[ii][jj].plot([meannI[0],meannI[0]+dirvecI[0,1]],[meanmI[0],meanmI[0]+dirvecI[1,1]],color='b',linestyle='--',linewidth=1.5)

# mE,nE=meigvecloading[:NE,0].copy(),neigvecloading[:NE,0].copy()
# confidence_ellipse(np.real(nE),np.real(mE),ax[ii][jj],edgecolor='orange')
# ax[ii][jj].scatter(nE,mE,s=10,c='red',alpha=0.5)
# mI,nI=meigvecloading[NE:,0].copy(),neigvecloading[NE:,0].copy()
# confidence_ellipse(np.real(nI),np.real(mI),ax[ii][jj],edgecolor='cyan')
# ax[ii][jj].scatter(nI,mI,s=10,c='blue',alpha=0.5)
# ax[ii][jj].axvline(c='grey', lw=1)
# ax[ii][jj].axhline(c='grey', lw=1)


# ax[ii][jj].set_xlim(xlims)
# ax[ii][jj].set_ylim(ylims)
# ax[ii][jj].set_xticks(xticks)
# ax[ii][jj].set_yticks(yticks)
# ax[ii][jj].set_aspect('equal')
fig,ax=plt.subplots(figsize=(4,4))
ax.scatter(np.real(eigvJ),np.imag(eigvJ),s=5.0,c='blue',alpha=0.5)

eigenvalue outlier: [[1.99148606]]
(2, 2)
(2, 2)


## mean connectivity's degree of symmetry

In [45]:
''' generate mean matrix ($\bar{J}^0$) '''
nrank=1
Nt=np.array([1000,1000])
NE,NI=Nt[0],Nt[1]
Nparams=np.array([NE,NI])
JE,JI,a,b=1.0,3.5,-0.6,0.6 # already recording -3.5
a,b=0.0,0.0
JEE,JIE,JEI,JII=JE+a,JE-a,JI-b,JI+b
# generate mean Am
Am,Jsv=generate_meanmat_eig(Nparams,JEE,JIE,JEI,JII)
# calculate the original \bar{m} and \bar{n}
eigvAm,eigveclAm,eigvecrAm=scpla.eig(Am,left=True,right=True)
print('original outlier:',eigvAm[0])
meigvecAm=eigvecrAm.copy()
neigvecAm=eigveclAm.copy()
for i in range(nrank):
    normalizecoe=(np.sum(neigvecAm[:,i]*meigvecAm[:,i]))
    neigvecAm[:,i]*=(eigvAm[i]/normalizecoe)
overlaporg = (neigvecAm.T@meigvecAm)
''' symmetry is applied to \bar{J} '''
symetaseries = np.linspace(0.0,1.0,11)
nrank,nrankJm,neta=1,2,len(symetaseries)

eigvalJmseries,mvecJmseries,nvecJmseries=np.zeros((neta,nrankJm)),np.zeros((neta,NE*2,nrankJm)),np.zeros((neta,NE*2,nrankJm))
overlapJmseries=np.zeros((neta,nrankJm,nrankJm))
eigvalOverlapJmseries=np.zeros((neta,nrankJm))
# generate symmetric random
gaverage=1.0
# # random 
# while(1):
#     grand = gmatamplitude_eig(gaverage**2,2)
#     idxeff = np.where(grand>0)
#     if (np.shape(idxeff)[1]>3):
#         np.random.shuffle(grand)
#         break
# print('random amp:',grand)
''' homogeneous case '''
gmat = gaverage**2/2*np.ones((2,2))
eigvgm,eigvecgm=la.eig(gmat)
radius_g = np.sqrt(eigvgm[0])
print('radius-g:',radius_g)
for idxeta, eta in enumerate(symetaseries):
    '''
    noise correlation of m and n change with random gain $\bar{g}$ and also
    symmetricity $\eta$
    '''
    asqr=(1+np.sqrt(1-eta**2))/2.0
    aamp=np.sqrt(asqr)  
    Jm0=Am.copy()
    Jm=aamp*Jm0.T+np.sqrt(1-aamp**2)*Jm0
    eigvJm,eigveclJm,eigvecrJm=scpla.eig(Jm,left=True,right=True)
    meigvecJm=eigvecrJm.copy()
    neigvecJm=eigveclJm.copy()
    for i in range(nrankJm):
        normalizecoe=(np.sum(neigvecJm[:,i]*meigvecJm[:,i]))
        neigvecJm[:,i]*=(eigvJm[i]/normalizecoe)
    eigvalJmseries[idxeta,:]=eigvJm[:nrankJm]
    mvecJmseries[idxeta,:,:]=meigvecJm[:,:nrankJm]
    nvecJmseries[idxeta,:,:]=neigvecJm[:,:nrankJm]
    overlapJmseries[idxeta,:,:]=np.squeeze(nvecJmseries[idxeta,:,:]).T@np.squeeze(mvecJmseries[idxeta,:,:])
    eigvalOverlapJmseries[idxeta,:],_=la.eig(np.squeeze(overlapJmseries[idxeta,:,:]))
    print('sym eigval:')
    print(eigvalJmseries[idxeta,:])
    print('overlap:')
    print(eigvalOverlapJmseries[idxeta,:])


    # Xinit=np.random.randn(NE*2,NE*2)/np.sqrt(NE)
    # X0=Xinit.copy()#*gaverage#/np.sqrt(2)#1.0/np.sqrt(2)
    # X=aamp*X0.T+np.sqrt(1-aamp**2)*X0



original outlier: (-2.499999999999994+0j)
radius-g: 1.0
sym eigval:
[-2.50000000e+00  3.48527964e-15]
overlap:
[-2.5        -0.46457775]
sym eigval:
[-2.96366045  0.34163833]
overlap:
[-2.96366045  0.34163833]
sym eigval:
[-3.34414816  0.60553537]
overlap:
[-3.34414816  0.60553537]
sym eigval:
[-3.67660774  0.82616918]
overlap:
[-3.67660774  0.82616918]
sym eigval:
[-3.97651869  1.0184788 ]
overlap:
[-3.97651869  1.0184788 ]
sym eigval:
[-4.25237372  1.19051154]
overlap:
[-4.25237372  1.19051154]
sym eigval:
[-4.50944892  1.34717126]
overlap:
[-4.50944892  1.34717126]
sym eigval:
[-4.75129864  1.49169744]
overlap:
[-4.75129864  1.49169744]
sym eigval:
[-4.98045831  1.62635635]
overlap:
[-4.98045831  1.62635635]
sym eigval:
[-5.19881535  1.75280316]
overlap:
[-5.19881535  1.75280316]
sym eigval:
[-5.4078219   1.87228799]
overlap:
[-5.4078219   1.87228799]


In [37]:
eigvalOverlapJmseries=np.zeros((neta,nrankJm))
ampsumOverlap=np.zeors(neta)
for idxeta, eta in enumerate(symetaseries):
    # overlapJmseries[idxeta,:,:]=np.squeeze(nvecJmseries[idxeta,:,:]).T@np.squeeze(mvecJmseries[idxeta,:,:])
    eigvalOverlapJmseries[idxeta,:],_=la.eig(np.squeeze(overlapJmseries[idxeta,:,:]))
    ampsumOverlap[idxeta]=

In [44]:
## change of eigenvalue?
fig,ax=plt.subplots(figsize=(12,3))
ax.plot(symetaseries,eigvalJmseries[:,0]-eigvAm[0],'r')
ax.plot(symetaseries,-eigvalJmseries[:,1],'b--')
ax.plot(symetaseries,eigvAm[0]*np.ones(neta),'k',linestyle='--')
# fig,ax=plt.subplots(figsize=(12,3))
# ax.plot(symetaseries,np.sum((neigvecAm.T*meigvecAm))*np.ones(neta),'k',linestyle='--')
# ax.plot(symetaseries,eigvalOverlapJmseries[:,0],'k')

# ax.plot(symetaseries,overlapJmseries[:,0,0],'r')
# ax.plot(symetaseries,overlapJmseries[:,0,1],color='orange')
# ax.plot(symetaseries,overlapJmseries[:,1,1],'b')
# ax.plot(symetaseries,overlapJmseries[:,1,0],color='cyan')




## $g_{\rm{mat}}$ and $\eta$, effects on eigen spectrum

In [267]:
idxeta=np.array([0,4,9])
print(symetaseries[idxeta])

[0.  0.4 0.9]


In [100]:
xticks = np.linspace(-4.0,3.0,5)
xlims = [-4.1,3.1]
yticks = np.linspace(-1.3,1.3,3)
ylims = [-1.31,1.31]
theta = np.linspace(0, 2 * np.pi, 200)
fig,ax=plt.subplots(3,3,figsize=(18,6))
figs,axs=plt.subplots(3,3,figsize=(18,6))
idxxx=0
# strfile='2021-02-02-17_07_43_heterNotrEEsymmetricity_data'
# # strfile='2021-01-29-05_51_27_symmetricity_data'
# strfile='2021-02-02-13_57_13_heterrIIsymmetricity_data'
strfile='2021-02-04-21_37_42_homogeneoussymmetricity_data'
data = np.load(strfile+'.npz')#'2021-01-19-12_02_53_heterogeneousgII_cloudsdata.npz')
nrank=1
# regenerate unperturbed low rank
Jparameters=data['Jparameters']
NE = int(Jparameters[4])
P = np.zeros((2,2))
P = np.array([[Jparameters[0],-Jparameters[1]],[Jparameters[2],-Jparameters[3]]])
Am=np.zeros((NE*2,NE*2))
Am[:NE,:NE],Am[:NE,NE:]=P[0,0]/NE,P[0,1]/NE
Am[NE:,:NE],Am[NE:,NE:]=P[1,0]/NE,P[1,1]/NE
eigvAm,eigveclAm,eigvecrAm=scpla.eig(Am,left=True,right=True)

# perturbed
mvecseries,nvecseries=data['mvecseries'],data['nvecseries']
eigvalseries=data['eigvalseries']
eigvAm=data['eigvAm']
gaverageseries=data['gaverageseries']
symetaseries=data['symetaseries']
''' how eigenvalue outlier changes with g_bar '''
# three eta 0.2,0.5,0.8
# np.zeros((ngavg,neta,ntrial,nrank))
ngavg,neta,ntrial,N,nrank=np.shape(mvecseries)
NE=int(N/2)
idxeta,itrial=np.array([0,4,9]),6#2#5#8#heter1,9
idxgavg=9#18#12#3#heter1,9
meigvec=mvecseries[:,:,:,:,:].copy()
neigvec=nvecseries[:,:,:,:,:].copy()

meanmE,meanmI,meannE,meannI=np.zeros(nrank),np.zeros(nrank),np.zeros(nrank),np.zeros(nrank)
# recording direction and shape
shapeamp=np.zeros((ngavg,neta,ntrial,2,2*nrank))
directionrot=np.zeros((ngavg,neta,ntrial,2,nrank*2))# 2 EI
meigvecloading,neigvecloading=meigvec.copy(),neigvec.copy()
scale_std=3.0
symetaseries=data['symetaseries']
idxeta = np.array([0,4,9])#[int(neta/2),neta-1])
# generate symmetric random
gaverage=1.0
for idd,index in enumerate(idxeta ):   
    eta=symetaseries[index]
    asqr=(1+np.sqrt(1-eta**2))/2.0
    aamp=np.sqrt(asqr)  
    Xinit=np.random.randn(NE*2,NE*2)/np.sqrt(NE)
    X0=Xinit.copy()#*gaverage#/np.sqrt(2)#1.0/np.sqrt(2)
    X=aamp*X0.T+np.sqrt(1-aamp**2)*X0
    # # ''' homogeneous '''
    # X*=(gaverage/np.sqrt(2))
    # # ''' heterogeneous rII '''
    # X[:NE,:NE]=0
    # X[:NE,NE:]=0
    # X[NE:,:NE]=0
    # X[NE:,NE:]*=np.sqrt(2)*gaverage
    # ''' heterogeneous not rEE '''
    X[:NE,:NE]=0
    X[:NE,NE:]*=(np.sqrt(2.0/3.0)*gaverage)
    X[NE:,:NE]*=(np.sqrt(2.0/3.0)*gaverage)
    X[NE:,NE:]*=(np.sqrt(2.0/3.0)*gaverage)
    # # random 
    # X[:NE,:NE]*=grand[0]
    # X[:NE,NE:]*=grand[1]
    # X[NE:,:NE]*=grand[2]
    # X[NE:,NE:]*=grand[3]
    # theogrand = (grand[1]*grand[2]*2+grand[0]**2+grand[3]**2)/2
    # ''' HOMO '''
    # gmat = np.ones((2,2))*gaverage**2/2
    # eigvgm,eigvecgm=la.eig(gmat)
    # eigvgmmax=np.max(eigvgm)
    # radius_g = np.sqrt(eigvgmmax)#np.sqrt(eigvgm[1])
    ''' heterogeneous NotrEE '''
    gmat = np.ones((2,2))*(np.sqrt(2.0/3.0)*gaverage)**2
    gmat[0,0]=0
    eigvgm,eigvecgm=la.eig(gmat)
    eigvgmmax=np.max(eigvgm)
    radius_g = np.sqrt(eigvgmmax)#np.sqrt(eigvgm[1])
    # ''' heterogeneous rII ''' 
    # gmat = np.zeros((2,2))
    # gmat[1,1]=(np.sqrt(2)*gaverage)**2
    # eigvgm,eigvecgm=la.eig(gmat)
    # eigvgmmax=np.max(eigvgm)
    # radius_g = np.sqrt(eigvgmmax)#np.sqrt(eigvgm[1])

    print(radius_g)
    longaxis,shortaxis=(1+eta)*radius_g ,(1-eta)*radius_g 
    xr = longaxis*np.cos(theta)
    yr = shortaxis*np.sin(theta)
    symmetricityX=np.sum(np.sum(X.T*X))/(NE*2)
    print(symmetricityX,eta)
    # overall
    J = X.copy()+Am.copy()
    # properties
    eigvJ,eigrvecJ=la.eig(J)
    inveigrvecJ=la.inv(eigrvecJ)
    
    real_eigsbulk,imag_eigsbulk=np.real(eigvJ[nrank:]),np.imag(eigvJ[nrank:])
    # # np.zeros((ngavg,neta,ntrial,nrank))
    # # idxgavg,idxeta,itrial=9,9,6#2#5#8#heter1,9
    # ax[idd][idxxx].scatter(np.real(eigvalseries[idxgavg,index,:,0]),np.imag(eigvalseries[idxgavg,index,:,0]),s=5,color='blue',alpha=0.5)
    # ax[idd][idxxx].scatter(real_eigsbulk,imag_eigsbulk,s=3,color='blue',alpha=0.25)
    # ax[idd][idxxx].scatter(np.real(eigvJ[:nrank]),np.imag(eigvJ[:nrank]),marker='^',s=5,color='red',alpha=1.0)
    # # xa = gaverage*np.cos(theta)
    # # ya = gaverage*np.sin(theta)
    # # ax[0].plot(xa, ya, color="darkred", linewidth=1.5,label=r'$r_g$')
    # ax[idd][idxxx].plot(xr, yr, color="darkred", linewidth=1.5,linestyle='--',label=r'ellipse')
    # # ax[0].scatter(np.real(eigvalAm[:nrank]),np.imag(eigvalAm[:nrank]),marker='^',s=15,color='red',alpha=1.0)
    # ax[idd][idxxx].set_xlim(xlims)
    # ax[idd][idxxx].set_ylim(ylims)
    # ax[idd][idxxx].set_xticks(xticks)
    # ax[idd][idxxx].set_yticks(yticks)
    # ax[idd][idxxx].set_aspect('equal')
    # ax[idd][idxxx].legend()
    # ax[idd][idxxx].grid()
    # ax[idd][idxxx].set_aspect('equal')
    # ''' histogram '''
    # nbins=np.linspace(-4.0,3.0,100)
    # n, bins, patches = axs[idd][idxxx].hist(np.real(eigvJ[:]), bins=nbins, density=True, color='gray',alpha=0.75)



       


1.0385996947011218
0.10015005563584367 0.1
1.0385996947011218
0.5010468875852336 0.5
1.0385996947011218
1.0010606852498745 1.0


In [21]:
xticks = np.linspace(-4.0,3.0,5)
xlims = [-4.1,3.1]
yticks = np.linspace(-1.3,1.3,3)
ylims = [-1.31,1.31]
for idd in range(3):
    for idxxx in range(3):
        ax[idd][idxxx].set_xlim(xlims)
        ax[idd][idxxx].set_ylim(ylims)
        ax[idd][idxxx].set_xticks(xticks)
        ax[idd][idxxx].set_yticks(yticks)

In [99]:
xticks = np.linspace(-0.3,0.4,2)
xlims = [-0.31,0.41]
yticks = np.linspace(-0.1,0.1,3)
ylims = [-0.11,0.11]
''' ------------------- loading space -------------------- '''
# figs2,axs2=plt.subplots(figsize=(5,5))
eigvalseries = data['eigvalseries']
# perturbed
mvecseries,nvecseries=data['mvecseries'],data['nvecseries']
eigvAm=data['eigvAm']
''' how eigenvalue outlier changes with g_bar '''
# three eta 0.2,0.5,0.8
# np.zeros((ngavg,neta,ntrial,nrank))
ngavg,neta,ntrial,N,nrank=np.shape(mvecseries)
# thetamaxseries=data['thetamaxseries']
scale_std=3.0
idxxsymeta=np.array([0,4,9])#idxeta.copy()
idxgavg=9
for didxeta, idxeta in enumerate(idxxsymeta):   
    ktrial=6
    meanmE,meanmI,meannE,meannI=np.zeros(nrank),np.zeros(nrank),np.zeros(nrank),np.zeros(nrank)
    meigvecloading,neigvecloading=mvecseries[idxgavg,idxeta,ktrial,:,0].copy(),nvecseries[idxgavg,idxeta,ktrial,:,0].copy()
    # if didxeta==1:
    #     meigvecloading,neigvecloading=-meigvecloading,-neigvecloading
    meigvecloading,neigvecloading=np.reshape(meigvecloading,(2*NE,1)),np.reshape(neigvecloading,(2*NE,1))
    # m vector
    # puring E
    mEmat = np.squeeze(meigvecloading[:NE,0])
    meanmEtotal,stdmEtotal = np.mean(mEmat),np.std(mEmat)
    varmE = mEmat - meanmEtotal
    idxwhere = np.where(np.abs(varmE)>scale_std*stdmEtotal)
    mEmat[idxwhere]=meanmEtotal
    meigvecloading[:NE,0]=mEmat
    meanmE[0]=np.mean(mEmat)

    # puring I
    mImat = np.squeeze(meigvecloading[NE:,0])
    meanmItotal,stdmItotal = np.mean(mImat),np.std(mImat)
    varmI = mImat - meanmItotal
    idxwhere = np.where(np.abs(varmI)>scale_std*stdmItotal)
    mImat[idxwhere]=meanmItotal
    meigvecloading[NE:,0]=mImat
    meanmI[0]=np.mean(mImat)
    
    # n vector
    # puring E
    nEmat = np.squeeze(neigvecloading[:NE,0])
    meannEtotal,stdnEtotal = np.mean(nEmat),np.std(nEmat)
    varnE = nEmat - meannEtotal
    idxwhere = np.where(np.abs(varnE)>scale_std*stdnEtotal)
    nEmat[idxwhere]=meannEtotal
    neigvecloading[:NE,0]=nEmat
    meannE[0]=np.mean(nEmat)

    # puring I
    nImat = np.squeeze(neigvecloading[NE:,0])
    meannItotal,stdnItotal = np.mean(nImat),np.std(nImat)
    varnI = nImat - meannItotal
    idxwhere = np.where(np.abs(varnI)>scale_std*stdnItotal)
    nImat[idxwhere]=meannItotal
    neigvecloading[NE:,0]=nImat
    meannI[0]=np.mean(nImat)

    ''' calculate directions '''
    noiseE,noiseI = np.zeros((NE,2,nrank)),np.zeros((NE,2,nrank))
    dirvecE,dirvecI=np.zeros((2,2)),np.zeros((2,2))
    for irank in range(nrank):
        # E 0 M 1 N
        noiseE[:,0,irank],noiseE[:,1,irank]= np.squeeze(neigvecloading[:NE,irank])-meannE[irank],np.squeeze(meigvecloading[:NE,irank])-meanmE[irank]
        noiseI[:,0,irank],noiseI[:,1,irank]= np.squeeze(neigvecloading[NE:,irank])-meannI[irank],np.squeeze(meigvecloading[NE:,irank])-meanmI[irank]
    # m1n1
    covdirE,covdirI=np.squeeze(noiseE[:,:,0]).T@np.squeeze(noiseE[:,:,0]),np.squeeze(noiseI[:,:,0]).T@np.squeeze(noiseI[:,:,0])
    _,dirvecE=la.eig(covdirE)
    thetamaxseries[idxgavg,idxeta,ktrial,0,0],thetamaxseries[idxgavg,idxeta,ktrial,0,1]  =np.arctan2(dirvecE[1,0],dirvecE[0,0]),np.arctan2(dirvecE[1,1],dirvecE[0,1])
    _,dirvecI=la.eig(covdirI)  
    thetamaxseries[idxgavg,idxeta,ktrial,1,0],thetamaxseries[idxgavg,idxeta,ktrial,1,1]  =np.arctan2(dirvecI[1,0],dirvecI[0,0]),np.arctan2(dirvecI[1,1],dirvecI[0,1])

    fig,ax=plt.subplots(figsize=(5,3))
    ax.scatter(neigvecloading[NE:,0],meigvecloading[NE:,0],s=5.0,c='blue',alpha=0.5)
    ax.scatter(neigvecloading[:NE,0],meigvecloading[:NE,0],s=5.0,c='red',alpha=0.5)

    ax.plot([meannE[0],meannE[0]+dirvecE[0,0]],[meanmE[0],meanmE[0]+dirvecE[1,0]],color='r',linestyle='--',linewidth=1.5)
    ax.plot([meannE[0],meannE[0]+dirvecE[0,1]],[meanmE[0],meanmE[0]+dirvecE[1,1]],color='r',linestyle='--',linewidth=1.5)
    ax.plot([meannI[0],meannI[0]+dirvecI[0,0]],[meanmI[0],meanmI[0]+dirvecI[1,0]],color='b',linestyle='--',linewidth=1.5)
    ax.plot([meannI[0],meannI[0]+dirvecI[0,1]],[meanmI[0],meanmI[0]+dirvecI[1,1]],color='b',linestyle='--',linewidth=1.5)

    mE,nE=meigvecloading[:NE,0].copy(),neigvecloading[:NE,0].copy()
    confidence_ellipse(np.real(nE),np.real(mE),ax,edgecolor='orange')
    ax.scatter(nE,mE,s=10,c='red',alpha=0.5)
    mI,nI=meigvecloading[NE:,0].copy(),neigvecloading[NE:,0].copy()
    confidence_ellipse(np.real(nI),np.real(mI),ax,edgecolor='cyan')
    ax.scatter(nI,mI,s=10,c='blue',alpha=0.5)
    ax.axvline(c='grey', lw=1)
    ax.axhline(c='grey', lw=1)
    xxx=np.linspace(0.0,1.0,10)
    # yyy=np.tan(thetamaxseries[idxeta,ktrial,0,0])*xxx
    # ax.plot(xxx,yyy,color='green',linestyle='--',linewidth=2.0)
    ax.set_xlim(xlims)
    ax.set_ylim(ylims)
    ax.set_xticks(xticks)
    ax.set_yticks(yticks)
    ax.set_aspect('equal')


KeyError: 'thetamaxseries is not a file in the archive'

## indirect relationship $\eta$-$\rho$(pearson] and $\eta$-$\lambda$

In [280]:
xticks = np.linspace(-4.0,3.0,5)
xlims = [-4.1,3.1]
yticks = np.linspace(-1.3,1.3,3)
ylims = [-1.31,1.31]
theta = np.linspace(0, 2 * np.pi, 200)
idxxx=0
# strfile='2021-02-04-21_37_42_homogeneoussymmetricity_data'
# strfile='2021-02-09-14_36_53_homogeneoussym_method2_data'
# strfile='2021-02-09-17_20_22_homogeneoussymNeg_method2_data'
# strfile='2021-02-09-16_28_19_homogeneoussym_method2_data'
# strfile='2021-02-09-16_43_40_homogeneoussym_method2_data'
strfile='2021-02-09-18_11_17_heterrII_method2_data'

data = np.load(strfile+'.npz')#'2021-01-19-12_02_53_heterogeneousgII_cloudsdata.npz')
nrank=1
# regenerate unperturbed low rank
Jparameters=data['Jparameters']
NE = int(Jparameters[4])

# perturbed
mvecseries,nvecseries=data['mvecseries'],data['nvecseries']
eigvalseries=data['eigvalseries']
eigvAm=data['eigvAm']
symetaseries=data['symetaseries']
''' how eigenvalue outlier changes with g_bar '''
# three eta 0.2,0.5,0.8
# np.zeros((ngavg,neta,ntrial,nrank))
neta,ntrial,N,nrank=np.shape(mvecseries)
NE=int(N/2)

homopearsoncorr_rho=np.zeros((neta,ntrial,2,nrank))
homothetamaxseries=np.zeros((neta,ntrial,2,2))
homoeigtotalseries=np.zeros((neta,ntrial,nrank))

''' get eigenvalue outlier series  '''
homoeigtotalseries[:,:,:]=eigvalseries
for idxeta, eta in enumerate(symetaseries):
    for ktrial in range(ntrial):
        meanmE,meanmI,meannE,meannI=np.zeros(nrank),np.zeros(nrank),np.zeros(nrank),np.zeros(nrank)
        # print(jdxgavg)
        meigvecloading,neigvecloading=mvecseries[idxeta,ktrial,:,0].copy(),nvecseries[idxeta,ktrial,:,0].copy()
        meigvecloading,neigvecloading=np.reshape(meigvecloading,(2*NE,1)),np.reshape(neigvecloading,(2*NE,1))
        scale_std=3.0
        for irank in range(nrank):
            # m vector
            # puring E
            mEmat = np.squeeze(meigvecloading[:NE,irank])
            meanmEtotal,stdmEtotal = np.mean(mEmat),np.std(mEmat)
            varmE = mEmat - meanmEtotal
            idxwhere = np.where(np.abs(varmE)>scale_std*stdmEtotal)
            mEmat[idxwhere]=meanmEtotal
            meigvecloading[:NE,irank]=mEmat
            meanmE[irank]=np.mean(mEmat)

            # puring I
            mImat = np.squeeze(meigvecloading[NE:,irank])
            meanmItotal,stdmItotal = np.mean(mImat),np.std(mImat)
            varmI = mImat - meanmItotal
            idxwhere = np.where(np.abs(varmI)>scale_std*stdmItotal)
            mImat[idxwhere]=meanmItotal
            meigvecloading[NE:,irank]=mImat
            meanmI[irank]=np.mean(mImat)
            
            # n vector
            # puring E
            nEmat = np.squeeze(neigvecloading[:NE,irank])
            meannEtotal,stdnEtotal = np.mean(nEmat),np.std(nEmat)
            varnE = nEmat - meannEtotal
            idxwhere = np.where(np.abs(varnE)>scale_std*stdnEtotal)
            nEmat[idxwhere]=meannEtotal
            neigvecloading[:NE,irank]=nEmat
            meannE[irank]=np.mean(nEmat)

            # puring I
            nImat = np.squeeze(neigvecloading[NE:,irank])
            meannItotal,stdnItotal = np.mean(nImat),np.std(nImat)
            varnI = nImat - meannItotal
            idxwhere = np.where(np.abs(varnI)>scale_std*stdnItotal)
            nImat[idxwhere]=meannItotal
            neigvecloading[NE:,irank]=nImat
            meannI[irank]=np.mean(nImat)

        ''' calculate directions '''
        noiseE,noiseI = np.zeros((NE,2,nrank)),np.zeros((NE,2,nrank))
        dirvecE,dirvecI=np.zeros((2,2)),np.zeros((2,2))
        for irank in range(nrank):
            # E 0 M 1 N
            noiseE[:,0,irank],noiseE[:,1,irank]= np.squeeze(neigvecloading[:NE,irank])-meannE[irank],np.squeeze(meigvecloading[:NE,irank])-meanmE[irank]
            noiseI[:,0,irank],noiseI[:,1,irank]= np.squeeze(neigvecloading[NE:,irank])-meannI[irank],np.squeeze(meigvecloading[NE:,irank])-meanmI[irank]
        # m1n1
        covdirE,covdirI=np.squeeze(noiseE[:,:,0]).T@np.squeeze(noiseE[:,:,0]),np.squeeze(noiseI[:,:,0]).T@np.squeeze(noiseI[:,:,0])
        _,dirvecE=la.eig(covdirE)
        homothetamaxseries[idxeta,ktrial,0,0],homothetamaxseries[idxeta,ktrial,0,1] =np.arctan2(dirvecE[1,0],dirvecE[0,0]),np.arctan2(dirvecE[1,1],dirvecE[0,1])
        _,dirvecI=la.eig(covdirI)  
        homothetamaxseries[idxeta,ktrial,1,0],homothetamaxseries[idxeta,ktrial,1,1]  =np.arctan2(dirvecI[1,0],dirvecI[0,0]),np.arctan2(dirvecI[1,1],dirvecI[0,1])
        ## pearson correlation 
        normmE,normmI,normnE,normnI=np.reshape(noiseE[:,0,0],(NE,1)),np.reshape(noiseI[:,0,0],(NE,1)),np.reshape(noiseE[:,1,0],(NE,1)),np.reshape(noiseI[:,1,0],(NE,1))
        normmE,normmI,normnE,normnI=normmE/la.norm(normmE),normmI/la.norm(normmI),normnE/la.norm(normnE),normnI/la.norm(normnI)
        homopearsoncorr_rho[idxeta,ktrial,0,0],homopearsoncorr_rho[idxeta,ktrial,1,0]=normmE.T@normnE,normmI.T@normnI

# homopearsoncorr_rho=np.zeros((ngavg,neta,ntrial,2,nrank))
# homothetamaxseries=np.zeros((ngavg,neta,ntrial,2,2))
# homoeigtotalseries=np.zeros((ngavg,neta,ntrial,nrank))

# ''' get eigenvalue outlier series  '''
# homoeigtotalseries[:,:,:,:]=eigvalseries

# for jdxgavg, gavg in enumerate(gaverageseries):
#     for idxeta, eta in enumerate(symetaseries):
#         for ktrial in range(ntrial):
#             meanmE,meanmI,meannE,meannI=np.zeros(nrank),np.zeros(nrank),np.zeros(nrank),np.zeros(nrank)
#             # print(jdxgavg)
#             meigvecloading,neigvecloading=mvecseries[jdxgavg,idxeta,ktrial,:,0].copy(),nvecseries[jdxgavg,idxeta,ktrial,:,0].copy()
#             meigvecloading,neigvecloading=np.reshape(meigvecloading,(2*NE,1)),np.reshape(neigvecloading,(2*NE,1))
#             scale_std=3.0
#             for irank in range(nrank):
#                 # m vector
#                 # puring E
#                 mEmat = np.squeeze(meigvecloading[:NE,irank])
#                 meanmEtotal,stdmEtotal = np.mean(mEmat),np.std(mEmat)
#                 varmE = mEmat - meanmEtotal
#                 idxwhere = np.where(np.abs(varmE)>scale_std*stdmEtotal)
#                 mEmat[idxwhere]=meanmEtotal
#                 meigvecloading[:NE,irank]=mEmat
#                 meanmE[irank]=np.mean(mEmat)

#                 # puring I
#                 mImat = np.squeeze(meigvecloading[NE:,irank])
#                 meanmItotal,stdmItotal = np.mean(mImat),np.std(mImat)
#                 varmI = mImat - meanmItotal
#                 idxwhere = np.where(np.abs(varmI)>scale_std*stdmItotal)
#                 mImat[idxwhere]=meanmItotal
#                 meigvecloading[NE:,irank]=mImat
#                 meanmI[irank]=np.mean(mImat)
                
#                 # n vector
#                 # puring E
#                 nEmat = np.squeeze(neigvecloading[:NE,irank])
#                 meannEtotal,stdnEtotal = np.mean(nEmat),np.std(nEmat)
#                 varnE = nEmat - meannEtotal
#                 idxwhere = np.where(np.abs(varnE)>scale_std*stdnEtotal)
#                 nEmat[idxwhere]=meannEtotal
#                 neigvecloading[:NE,irank]=nEmat
#                 meannE[irank]=np.mean(nEmat)

#                 # puring I
#                 nImat = np.squeeze(neigvecloading[NE:,irank])
#                 meannItotal,stdnItotal = np.mean(nImat),np.std(nImat)
#                 varnI = nImat - meannItotal
#                 idxwhere = np.where(np.abs(varnI)>scale_std*stdnItotal)
#                 nImat[idxwhere]=meannItotal
#                 neigvecloading[NE:,irank]=nImat
#                 meannI[irank]=np.mean(nImat)

#             ''' calculate directions '''
#             noiseE,noiseI = np.zeros((NE,2,nrank)),np.zeros((NE,2,nrank))
#             dirvecE,dirvecI=np.zeros((2,2)),np.zeros((2,2))
#             for irank in range(nrank):
#                 # E 0 M 1 N
#                 noiseE[:,0,irank],noiseE[:,1,irank]= np.squeeze(neigvecloading[:NE,irank])-meannE[irank],np.squeeze(meigvecloading[:NE,irank])-meanmE[irank]
#                 noiseI[:,0,irank],noiseI[:,1,irank]= np.squeeze(neigvecloading[NE:,irank])-meannI[irank],np.squeeze(meigvecloading[NE:,irank])-meanmI[irank]
#             # m1n1
#             covdirE,covdirI=np.squeeze(noiseE[:,:,0]).T@np.squeeze(noiseE[:,:,0]),np.squeeze(noiseI[:,:,0]).T@np.squeeze(noiseI[:,:,0])
#             _,dirvecE=la.eig(covdirE)
#             homothetamaxseries[jdxgavg,idxeta,ktrial,0,0],homothetamaxseries[jdxgavg,idxeta,ktrial,0,1] =np.arctan2(dirvecE[1,0],dirvecE[0,0]),np.arctan2(dirvecE[1,1],dirvecE[0,1])
#             _,dirvecI=la.eig(covdirI)  
#             homothetamaxseries[jdxgavg,idxeta,ktrial,1,0],homothetamaxseries[jdxgavg,idxeta,ktrial,1,1]  =np.arctan2(dirvecI[1,0],dirvecI[0,0]),np.arctan2(dirvecI[1,1],dirvecI[0,1])
#             ## pearson correlation 
#             normmE,normmI,normnE,normnI=np.reshape(noiseE[:,0,0],(NE,1)),np.reshape(noiseI[:,0,0],(NE,1)),np.reshape(noiseE[:,1,0],(NE,1)),np.reshape(noiseI[:,1,0],(NE,1))
#             normmE,normmI,normnE,normnI=normmE/la.norm(normmE),normmI/la.norm(normmI),normnE/la.norm(normnE),normnI/la.norm(normnI)
#             homopearsoncorr_rho[jdxgavg,idxeta,ktrial,0,0],homopearsoncorr_rho[jdxgavg,idxeta,ktrial,1,0]=normmE.T@normnE,normmI.T@normnI
    
    # fig,ax=plt.subplots(figsize=(4,4))  
    # ax.scatter(neigvecloading[NE:,0],meigvecloading[NE:,0],s=5.0,c='blue',alpha=0.5)
    # ax.scatter(neigvecloading[:NE,0],meigvecloading[:NE,0],s=5.0,c='red',alpha=0.5)

    # # ax.plot([meannE[0],meannE[0]+dirvecE[0,0]],[meanmE[0],meanmE[0]+dirvecE[1,0]],color='r',linestyle='--',linewidth=1.5)
    # # ax.plot([meannE[0],meannE[0]+dirvecE[0,1]],[meanmE[0],meanmE[0]+dirvecE[1,1]],color='r',linestyle='--',linewidth=1.5)
    # # ax.plot([meannI[0],meannI[0]+dirvecI[0,0]],[meanmI[0],meanmI[0]+dirvecI[1,0]],color='b',linestyle='--',linewidth=1.5)
    # # ax.plot([meannI[0],meannI[0]+dirvecI[0,1]],[meanmI[0],meanmI[0]+dirvecI[1,1]],color='b',linestyle='--',linewidth=1.5)

    # mE,nE=meigvecloading[:NE,0].copy(),neigvecloading[:NE,0].copy()
    # confidence_ellipse(np.real(nE),np.real(mE),ax,edgecolor='orange')
    # ax.scatter(nE,mE,s=10,c='red',alpha=0.5)
    # mI,nI=meigvecloading[NE:,0].copy(),neigvecloading[NE:,0].copy()
    # confidence_ellipse(np.real(nI),np.real(mI),ax,edgecolor='cyan')
    # ax.scatter(nI,mI,s=10,c='blue',alpha=0.5)
    # ax.axvline(c='grey', lw=1)
    # ax.axhline(c='grey', lw=1)
    # # xxx=np.linspace(0.0,1.0,10)
    # # yyy=np.tan(thetamaxseries[iddg,jdxgavg,idxeta,6,0,0])*xxx
    # # ax.plot(xxx,yyy,color='green',linestyle='--',linewidth=2.0)
    # # ax.set_xlim(xlims)
    # # ax.set_ylim(ylims)
    # # ax.set_xticks(xticks)
    # # ax.set_yticks(yticks)
    # # ax.set_aspect('equal')
            


In [282]:
import mpl_toolkits.axisartist as axisartist
# #object ax
''' font setting '''
font_format = {'family':'Calibre', 'size':16}#:'Times New Roman', 'size':34}
## limit 
# xticks = np.linspace(0.0,1.0,2)
# xlims = [-.01,1.01]
# yticks = np.linspace(-1.0,0.0,2)
# ylims = [-1.01,0.01]
# delstr='bottom'
# signeigv=-1

xticks = np.linspace(0.0,1.0,2)
xlims = [-.01,1.01]
yticks = np.linspace(0.0,1.0,2)
ylims = [-.01,1.01]
delstr='top'
sigeigv=1

fig,ax=plt.subplots(1,2,figsize=(9,5))
i=0
eigtemp=np.zeros((neta+1,1))
pearsoncorrtemp=np.zeros((neta+1,2))
eigtemp[0,0]=eigvAm[0]
eigtemp[1:,0]=np.mean(np.squeeze(homoeigtotalseries[:,:,0]),axis=1)
pearsoncorrtemp[1:,0]=np.mean(np.squeeze(homopearsoncorr_rho[:,:,0,0]),axis=1)
pearsoncorrtemp[1:,1]=np.mean(np.squeeze(homopearsoncorr_rho[:,:,1,0]),axis=1)
symetaseries_=np.zeros(neta+1)
symetaseries_[1:]=symetaseries.copy()
ax[0].plot(symetaseries_,pearsoncorrtemp[:,0],color=Ecolor[i],linewidth=1.5,alpha=1,label=r'$E$ population')
ax[0].plot(symetaseries_,pearsoncorrtemp[:,1],color=Icolor[i],linewidth=1.5,alpha=1,label=r'$I$ population')
ax[0].scatter(symetaseries_,pearsoncorrtemp[:,0],s=20.0,c=Ecolor[i],marker='*',alpha=1)
ax[0].scatter(symetaseries_,pearsoncorrtemp[:,1],s=20.0,c=Icolor[i],marker='*',alpha=1)
ax[0].set_xlabel(r'$\eta$',font_format)
ax[0].set_ylabel(r'$\rho$[pearson]',font_format)
ax[0].spines[delstr].set_color('none')
ax[0].spines['right'].set_color('none')


ax[1].plot(symetaseries_,sigeigv*eigtemp/eigvAm[0],color='black',linewidth=1.5,alpha=1)
ax[1].scatter(symetaseries_,sigeigv*eigtemp/eigvAm[0],s=20.0,c='k',marker='*',alpha=1)
ax[1].set_xlabel(r'$\eta$',font_format)
ax[1].set_ylabel(r'$\lambda/\vert\lambda^O\vert$',font_format)
ax[1].spines['bottom'].set_color('none')
ax[1].spines['right'].set_color('none')

for i in range(1):
    # ax[i].set_xlim(xlims)
    # ax[i].set_ylim(ylims)
    # ax[i].set_xticks([])#xticks)
    # ax[i].set_yticks([])#yticks)
    ax[i].set_aspect('equal')
    # ax[i].legend([r'$E$ population',r'$I$ population'],prop={'family' : 'calibre', 'size'   : 12})
for i in range(1,2,1):
    # ax[i].set_xlim(xlims)
    # ax[i].set_ylim([0.0,1.0])
    # ax[i].set_xticks([])#xticks)
    # ax[i].set_yticks([])#yticks)
    ax[i].set_aspect('equal')
    # ax[i].legend(font_format)


In [281]:
import mpl_toolkits.axisartist as axisartist
# #object ax
''' font setting '''
font_format = {'family':'Calibre', 'size':16}#:'Times New Roman', 'size':34}
## limit 
xticks = np.linspace(0.0,1.0,2)
xlims = [-.01,1.01]
yticks = np.linspace(-1.0,0.0,2)
ylims = [-1.01,0.01]

fig,ax=plt.subplots(1,2,figsize=(9,5))
i=0
eigtemp=np.zeros((neta+1,1))
pearsoncorrtemp=np.zeros((neta+1,2))
eigtemp[0,0]=eigvAm[0]
pearsoncorrtemp[1:,0]=np.mean(np.squeeze(homopearsoncorr_rho[-1,:,:,0,0]),axis=1)
pearsoncorrtemp[1:,1]=np.mean(np.squeeze(homopearsoncorr_rho[-1,:,:,1,0]),axis=1)
symetaseries_=np.zeros(neta+1)
symetaseries_[1:]=symetaseries.copy()
ax[0].plot(symetaseries_,pearsoncorrtemp[:,0],color=Ecolor[i],linewidth=1.5,alpha=1,label=r'$E$ population')
ax[0].plot(symetaseries_,pearsoncorrtemp[:,1],color=Icolor[i],linewidth=1.5,alpha=1,label=r'$I$ population')
ax[0].scatter(symetaseries_,pearsoncorrtemp[:,0],s=20.0,c=Ecolor[i],marker='*',alpha=1)
ax[0].scatter(symetaseries_,pearsoncorrtemp[:,1],s=20.0,c=Icolor[i],marker='*',alpha=1)
ax[0].set_xlabel(r'$\eta$',font_format)
ax[0].set_ylabel(r'$\rho$[pearson]',font_format)
ax[0].spines['bottom'].set_color('none')
ax[0].spines['right'].set_color('none')


ax[1].plot(symetaseries_,-eigtemp/eigvAm[0],color='black',linewidth=1.5,alpha=1)
ax[1].scatter(symetaseries_,-eigtemp/eigvAm[0],s=20.0,c='k',marker='*',alpha=1)
ax[1].set_xlabel(r'$\eta$',font_format)
ax[1].set_ylabel(r'$\lambda/\vert\lambda^O\vert$',font_format)
ax[1].spines['bottom'].set_color('none')
ax[1].spines['right'].set_color('none')

for i in range(1):
    # ax[i].set_xlim(xlims)
    # ax[i].set_ylim(ylims)
    # ax[i].set_xticks([])#xticks)
    # ax[i].set_yticks([])#yticks)
    ax[i].set_aspect('equal')
    ax[i].legend([r'$E$ population',r'$I$ population'],prop={'family' : 'calibre', 'size'   : 12})
for i in range(1,2,1):
    # ax[i].set_xlim(xlims)
    # ax[i].set_ylim([-2.0,-1.0])
    # ax[i].set_xticks([])#xticks)
    # ax[i].set_yticks([])#yticks)
    ax[i].set_aspect('equal')
    # ax[i].legend(font_format)


IndexError: too many indices for array

## Contonuous trendency

In [157]:
xticks = np.linspace(-4.0,3.0,5)
xlims = [-4.1,3.1]
yticks = np.linspace(-1.3,1.3,3)
ylims = [-1.31,1.31]
theta = np.linspace(0, 2 * np.pi, 200)
idxxx=0
# strfile='2021-02-02-17_07_43_heterNotrEEsymmetricity_data'
# strfile='2021-02-02-13_57_13_heterrIIsymmetricity_data'
strfile='2021-02-04-21_37_42_homogeneoussymmetricity_data'
data = np.load(strfile+'.npz')#'2021-01-19-12_02_53_heterogeneousgII_cloudsdata.npz')
nrank=1
# regenerate unperturbed low rank
Jparameters=data['Jparameters']
NE = int(Jparameters[4])

# perturbed
mvecseries,nvecseries=data['mvecseries'],data['nvecseries']
eigvalseries=data['eigvalseries']
eigvAm=data['eigvAm']
gaverageseries=data['gaverageseries']
symetaseries=data['symetaseries']
''' how eigenvalue outlier changes with g_bar '''
# three eta 0.2,0.5,0.8
# np.zeros((ngavg,neta,ntrial,nrank))
ngavg,neta,ntrial,N,nrank=np.shape(mvecseries)
NE=int(N/2)

# pearsoncorr_rho=np.zeros((3,ngavg,neta,ntrial,2,nrank))
# thetamaxseries=np.zeros((3,ngavg,neta,ntrial,2,2))
# eigtotalseries=np.zeros((3,ngavg,neta,ntrial,nrank))
iddg=2
''' get eigenvalue outlier series  '''
eigtotalseries[iddg,:,:,:,:]=eigvalseries

for jdxgavg, gavg in enumerate(gaverageseries):
    for idxeta, eta in enumerate(symetaseries):
        for ktrial in range(ntrial):
            meanmE,meanmI,meannE,meannI=np.zeros(nrank),np.zeros(nrank),np.zeros(nrank),np.zeros(nrank)
            # print(jdxgavg)
            meigvecloading,neigvecloading=mvecseries[jdxgavg,idxeta,ktrial,:,0].copy(),nvecseries[jdxgavg,idxeta,ktrial,:,0].copy()
            meigvecloading,neigvecloading=np.reshape(meigvecloading,(2*NE,1)),np.reshape(neigvecloading,(2*NE,1))
            scale_std=3.0
            for irank in range(nrank):
                # m vector
                # puring E
                mEmat = np.squeeze(meigvecloading[:NE,irank])
                meanmEtotal,stdmEtotal = np.mean(mEmat),np.std(mEmat)
                varmE = mEmat - meanmEtotal
                idxwhere = np.where(np.abs(varmE)>scale_std*stdmEtotal)
                mEmat[idxwhere]=meanmEtotal
                meigvecloading[:NE,irank]=mEmat
                meanmE[irank]=np.mean(mEmat)

                # puring I
                mImat = np.squeeze(meigvecloading[NE:,irank])
                meanmItotal,stdmItotal = np.mean(mImat),np.std(mImat)
                varmI = mImat - meanmItotal
                idxwhere = np.where(np.abs(varmI)>scale_std*stdmItotal)
                mImat[idxwhere]=meanmItotal
                meigvecloading[NE:,irank]=mImat
                meanmI[irank]=np.mean(mImat)
                
                # n vector
                # puring E
                nEmat = np.squeeze(neigvecloading[:NE,irank])
                meannEtotal,stdnEtotal = np.mean(nEmat),np.std(nEmat)
                varnE = nEmat - meannEtotal
                idxwhere = np.where(np.abs(varnE)>scale_std*stdnEtotal)
                nEmat[idxwhere]=meannEtotal
                neigvecloading[:NE,irank]=nEmat
                meannE[irank]=np.mean(nEmat)

                # puring I
                nImat = np.squeeze(neigvecloading[NE:,irank])
                meannItotal,stdnItotal = np.mean(nImat),np.std(nImat)
                varnI = nImat - meannItotal
                idxwhere = np.where(np.abs(varnI)>scale_std*stdnItotal)
                nImat[idxwhere]=meannItotal
                neigvecloading[NE:,irank]=nImat
                meannI[irank]=np.mean(nImat)

            ''' calculate directions '''
            noiseE,noiseI = np.zeros((NE,2,nrank)),np.zeros((NE,2,nrank))
            dirvecE,dirvecI=np.zeros((2,2)),np.zeros((2,2))
            for irank in range(nrank):
                # E 0 M 1 N
                noiseE[:,0,irank],noiseE[:,1,irank]= np.squeeze(neigvecloading[:NE,irank])-meannE[irank],np.squeeze(meigvecloading[:NE,irank])-meanmE[irank]
                noiseI[:,0,irank],noiseI[:,1,irank]= np.squeeze(neigvecloading[NE:,irank])-meannI[irank],np.squeeze(meigvecloading[NE:,irank])-meanmI[irank]
            # m1n1
            covdirE,covdirI=np.squeeze(noiseE[:,:,0]).T@np.squeeze(noiseE[:,:,0]),np.squeeze(noiseI[:,:,0]).T@np.squeeze(noiseI[:,:,0])
            _,dirvecE=la.eig(covdirE)
            thetamaxseries[iddg,jdxgavg,idxeta,ktrial,0,0],thetamaxseries[iddg,jdxgavg,idxeta,ktrial,0,1] =np.arctan2(dirvecE[1,0],dirvecE[0,0]),np.arctan2(dirvecE[1,1],dirvecE[0,1])
            _,dirvecI=la.eig(covdirI)  
            thetamaxseries[iddg,jdxgavg,idxeta,ktrial,1,0],thetamaxseries[iddg,jdxgavg,idxeta,ktrial,1,1]  =np.arctan2(dirvecI[1,0],dirvecI[0,0]),np.arctan2(dirvecI[1,1],dirvecI[0,1])
            ## pearson correlation 
            normmE,normmI,normnE,normnI=np.reshape(noiseE[:,0,0],(NE,1)),np.reshape(noiseI[:,0,0],(NE,1)),np.reshape(noiseE[:,1,0],(NE,1)),np.reshape(noiseI[:,1,0],(NE,1))
            normmE,normmI,normnE,normnI=normmE/la.norm(normmE),normmI/la.norm(normmI),normnE/la.norm(normnE),normnI/la.norm(normnI)
            pearsoncorr_rho[iddg,jdxgavg,idxeta,ktrial,0,0],pearsoncorr_rho[iddg,jdxgavg,idxeta,ktrial,1,0]=normmE.T@normnE,normmI.T@normnI
    
    fig,ax=plt.subplots(figsize=(4,4))  
    ax.scatter(neigvecloading[NE:,0],meigvecloading[NE:,0],s=5.0,c='blue',alpha=0.5)
    ax.scatter(neigvecloading[:NE,0],meigvecloading[:NE,0],s=5.0,c='red',alpha=0.5)

    # ax.plot([meannE[0],meannE[0]+dirvecE[0,0]],[meanmE[0],meanmE[0]+dirvecE[1,0]],color='r',linestyle='--',linewidth=1.5)
    # ax.plot([meannE[0],meannE[0]+dirvecE[0,1]],[meanmE[0],meanmE[0]+dirvecE[1,1]],color='r',linestyle='--',linewidth=1.5)
    # ax.plot([meannI[0],meannI[0]+dirvecI[0,0]],[meanmI[0],meanmI[0]+dirvecI[1,0]],color='b',linestyle='--',linewidth=1.5)
    # ax.plot([meannI[0],meannI[0]+dirvecI[0,1]],[meanmI[0],meanmI[0]+dirvecI[1,1]],color='b',linestyle='--',linewidth=1.5)

    mE,nE=meigvecloading[:NE,0].copy(),neigvecloading[:NE,0].copy()
    confidence_ellipse(np.real(nE),np.real(mE),ax,edgecolor='orange')
    ax.scatter(nE,mE,s=10,c='red',alpha=0.5)
    mI,nI=meigvecloading[NE:,0].copy(),neigvecloading[NE:,0].copy()
    confidence_ellipse(np.real(nI),np.real(mI),ax,edgecolor='cyan')
    ax.scatter(nI,mI,s=10,c='blue',alpha=0.5)
    ax.axvline(c='grey', lw=1)
    ax.axhline(c='grey', lw=1)
    # xxx=np.linspace(0.0,1.0,10)
    # yyy=np.tan(thetamaxseries[iddg,jdxgavg,idxeta,6,0,0])*xxx
    # ax.plot(xxx,yyy,color='green',linestyle='--',linewidth=2.0)
    # ax.set_xlim(xlims)
    # ax.set_ylim(ylims)
    # ax.set_xticks(xticks)
    # ax.set_yticks(yticks)
    # ax.set_aspect('equal')
            


(2, 2)
(2, 2)
(2, 2)
(2, 2)
(2, 2)
(2, 2)
(2, 2)
(2, 2)
(2, 2)
(2, 2)
(2, 2)
(2, 2)
(2, 2)
(2, 2)
(2, 2)
(2, 2)
(2, 2)
(2, 2)
(2, 2)
(2, 2)


In [146]:
# saving data
now = time.strftime("%Y-%m-%d-%H_%M_%S",time.localtime(time.time()))
np.savez(now+'_entirethetaeig_pearcorr_data',Jparameters=Jparameters,gparameters=gparameters,eigvAm=eigvAm,thetamaxseries=thetamaxseries,pearsoncorr_rho=pearsoncorr_rho,eigtotalseries=eigtotalseries)
# eigtotalseries=np.zeros((3,ngavg,neta,ntrial,nrank)))

In [181]:
strfile='2021-02-08-21_41_53_entirethetaeig_pearcorr_data'
data = np.load(strfile+'.npz')
thetamaxseries=data['thetamaxseries']
eigtotalseries=data['eigtotalseries']
pearsoncorr_rho=data['pearsoncorr_rho'] 
xticks = np.linspace(1.0,1.4,2)
xlims = [0.99,1.41]
yticks = np.linspace(-1.0,0.0,2)
ylims = [-1.01,0.01]
# thetatotalseries=np.zeros((neta,2))
# ## average trial
fig,ax=plt.subplots(1,2,figsize=(6,3))
Ecolor=['darkred','red','orange']
Icolor=['darkblue','blue','cyan']
for i in range(3): 
    eigtemp=np.zeros((neta+1,1))
    pearsoncorrtemp=np.zeros((neta+1,2))
    eigtemp[0,0]=eigvAm[0]
    eigtemp[1:,0]=np.mean(np.squeeze(eigtotalseries[i,-1,:,:,0]),axis=1)
    pearsoncorrtemp[1:,0]=np.mean(np.squeeze(pearsoncorr_rho[i,-1,:,:,0,0]),axis=1)
    pearsoncorrtemp[1:,1]=np.mean(np.squeeze(pearsoncorr_rho[i,-1,:,:,1,0]),axis=1)
    ax[0].plot(eigtemp/eigvAm[0],pearsoncorrtemp[:,0],color=Ecolor[i],linewidth=1.5,alpha=1)
    ax[1].plot(eigtemp/eigvAm[0],pearsoncorrtemp[:,1],color=Icolor[i],linewidth=1.5,alpha=1)
    ax[0].scatter(eigtemp/eigvAm[0],pearsoncorrtemp[:,0],s=20.0,c=Ecolor[i],marker='*',alpha=1)
    ax[1].scatter(eigtemp/eigvAm[0],pearsoncorrtemp[:,1],s=20.0,c=Icolor[i],marker='*',alpha=1)
for i in range(2):
    ax[i].set_xlim(xlims)
    ax[i].set_ylim(ylims)
    ax[i].set_xticks(xticks)
    ax[i].set_yticks(yticks)
    if ax[i].is_last_row():
        ax[i].set_xlabel(r'$[\lambda]$',fontsize=12)
    if ax[i].is_first_col():
        ax[i].set_ylabel(r'$\rho$(pearson)',fontsize=12)
plt.tight_layout()
# fig,ax=plt.subplots(2,1,figsize=(6,6)) 
# for i in range(3): 
#     ax[0].plot(np.mean(np.squeeze(eigtotalseries[i,-1,:,:,0]),axis=1),np.mean(np.squeeze(thetamaxseries[i,-1,:,:,1,0]),axis=1),color=clseries[i],alpha=0.5)
#     ax[1].plot(np.mean(np.squeeze(eigtotalseries[i,-1,:,:,0]),axis=1),np.mean(np.squeeze(thetamaxseries[i,-1,:,:,1,1]),axis=1),color=clseries[i],alpha=0.5)

In [137]:
eigmean,thetamean =np.zeros((3,neta)),np.zeros((3,neta))
for i in range(3): 
    eigmean[i,:]=np.mean(np.squeeze(eigtotalseries[i,-1,:,:,0]),axis=1)
print(eigmean[2,-1]/eigmean[0,-1],(2)**(1/4.0))

1.0628616526650683 1.189207115002721


## generalized eigenspectrum

In [24]:
# # generate mean matrix
# nrank=1
# Nt=np.array([1000,1000])
# NE,NI=Nt[0],Nt[1]
# Nparams=np.array([NE,NI])
# JE,JI,a,b=1.0,3.5,-0.6,0.6 # already recording -3.5
# a,b=0.0,0.0
# JEE,JIE,JEI,JII=JE+a,JE-a,JI-b,JI+b
# # generate mean Am
# Am,Jsv=generate_meanmat_eig(Nparams,JEE,JIE,JEI,JII)
# # calculate the original \bar{m} and \bar{n}
# eigvAm,eigveclAm,eigvecrAm=scpla.eig(Am,left=True,right=True)
# meigvecAm=eigvecrAm.copy()
# neigvecAm=eigveclAm.copy()
# for i in range(nrank):
#     normalizecoe=(np.sum(neigvecAm[:,i]*meigvecAm[:,i]))
#     neigvecAm[:,i]*=(eigvAm[i]/normalizecoe)
# print('normalization m & n')
# print(la.norm(meigvecAm[:,0]),la.norm(neigvecAm[:,0]))
# print('eigenvalues:',eigvAm[:nrank])

# '''
# fixed g_bar but r_g changes (randomly insert)
# eta changes and find out the relationships with eta, rotation and eigenvalue outliers
# * symetaseries 
# * gaverages =1.0
# * grand fixed 
# * ntrials
# '''
xticks = np.linspace(-4.0,3.0,5)
xlims = [-4.1,3.1]
yticks = np.linspace(-1.3,1.3,3)
ylims = [-1.31,1.31]
theta = np.linspace(0, 2 * np.pi, 200)
# fig,ax=plt.subplots(3,3,figsize=(24,6))
idxxx=1
strfile = '2021-02-06-15_08_30_hetersymmetricity_data'
# strfile='2021-02-06-21_26_05_hetersymmetricity_data'
# strfile='2021-02-06-16_02_02_hetersymmetricity_data'
# strfile='2021-02-06-13_31_42_hetersymmetricity_data'
data = np.load(strfile+'.npz')#'2021-01-19-12_02_53_heterogeneousgII_cloudsdata.npz')
# perturbed
symetaseries = np.linspace(0.0,1.0,11) 
mvecseries,nvecseries=data['mvecseries'],data['nvecseries']
eigvalseries=data['eigvalseries']
eigvAm=data['eigvAm']
# random gain and symmetricity
gaverage=1.0
idxxsymeta = np.array([0,4,9]) # a series of symmetricity value
nrank,ntrial,neta=1,1,len(symetaseries)
gparameters=data['gparameters']
grand=gparameters[:4]
''' test r_g and gbar'''
# theoretical M (grandom) matrix 
xee,xei,xie,xii=grand[0],grand[1],grand[2],grand[3]
gmat = np.array([[xee**2,xei**2],[xie**2,xii**2]])
# first do not multiply at
gaverage_=0
for i in range(2):
    for j in range(2):
        gaverage_+=gmat[i,j]/2 # ntype=2
gaverage_=np.sqrt(gaverage_)
eigvgm,eigvecgm=la.eig(gmat) 
r_g2=np.max(eigvgm)
r_g = np.sqrt(r_g2)
print('gbar:,',gaverage_,'; effective gain:',r_g)
for didxeta, idxeta in enumerate(idxxsymeta):
    eta=symetaseries[idxeta]
    asqr=(1+np.sqrt(1-eta**2))/2.0
    aamp=np.sqrt(asqr)
    for ktrial in range(ntrial):
        Xinit=np.random.randn(NE*2,NE*2)/np.sqrt(NE)
        X0=Xinit.copy()#*gaverage#/np.sqrt(2)#1.0/np.sqrt(2)
        X=aamp*X0.T+np.sqrt(1-aamp**2)*X0
        # # ''' homogeneous '''
        # X*=(gaverage/np.sqrt(2))
        # # ''' heterogeneous rII '''
        # X[:NE,:NE]=0
        # X[:NE,NE:]=0
        # X[NE:,:NE]=0
        # X[NE:,NE:]*=np.sqrt(2)*gaverage
        # # ''' heterogeneous not rEE '''
        # X[:NE,:NE]=0
        # X[:NE,NE:]*=(np.sqrt(2.0/3.0)*gaverage)
        # X[NE:,:NE]*=(np.sqrt(2.0/3.0)*gaverage)
        # X[NE:,NE:]*=(np.sqrt(2.0/3.0)*gaverage)
        # random 
        longaxis,shortaxis=(1+eta)*r_g ,(1-eta)*r_g 
        xr = longaxis*np.cos(theta)
        yr = shortaxis*np.sin(theta)
        X[:NE,:NE]*=grand[0]
        X[:NE,NE:]*=grand[1]
        X[NE:,:NE]*=grand[2]
        X[NE:,NE:]*=grand[3]
        theogrand = (grand[1]*grand[2]*2+grand[0]**2+grand[3]**2)/2
        symmetricityX=np.sum(np.sum(X.T*X))/(NE*2)
        print(symmetricityX,theogrand*eta)
        # overall
        J = X.copy()+Am.copy()
        # properties
        eigvJ,eigrvecJ=la.eig(J)
        inveigrvecJ=la.inv(eigrvecJ)
        meig = np.squeeze(eigrvecJ[:,:nrank].copy())
        meig = np.reshape(meig,(NE*2,nrank))
        neig = np.squeeze(inveigrvecJ[:nrank,:].copy()) # inverse
        neig = np.reshape(neig.T,(NE*2,nrank))

    real_eigsbulk,imag_eigsbulk=np.real(eigvJ[nrank:]),np.imag(eigvJ[nrank:])
    ax[didxeta][idxxx].scatter(real_eigsbulk,imag_eigsbulk,s=3,color='blue',alpha=0.25)
    ax[didxeta][idxxx].scatter(np.real(eigvalseries[idxeta,:,0]),np.imag(eigvalseries[idxeta,:,0]),s=5,color='blue',alpha=0.5)
    ax[didxeta][idxxx].scatter(np.real(eigvJ[:nrank]),np.imag(eigvJ[:nrank]),marker='^',s=5,color='red',alpha=1.0)
    # xa = gaverage*np.cos(theta)
    # ya = gaverage*np.sin(theta)
    # ax[0].plot(xa, ya, color="darkred", linewidth=1.5,label=r'$r_g$')
    ax[didxeta][idxxx].plot(xr, yr, color="darkred", linewidth=1.5,linestyle='--',label=r'ellipse')
    # ax[0].scatter(np.real(eigvalAm[:nrank]),np.imag(eigvalAm[:nrank]),marker='^',s=15,color='red',alpha=1.0)
    ax[didxeta][idxxx].set_xlim(xlims)
    ax[didxeta][idxxx].set_ylim(ylims)
    ax[didxeta][idxxx].set_xticks(xticks)
    ax[didxeta][idxxx].set_yticks(yticks)
    ax[didxeta][idxxx].set_aspect('equal')
    ax[didxeta][idxxx].legend()
    ax[didxeta][idxxx].grid()
    ax[didxeta][idxxx].set_aspect('equal')
print(grand)

gbar:, 1.0 ; effective gain: 1.2335679044368006
0.0009465889299874597 0.0
0.3994390064450738 0.3999501691625933
0.9001627314466651 0.8998878806158348
[1.21967081 0.47989088 0.46410626 0.25828906]


## relationships between eigenvalue outliers and rotation


In [133]:
eigout3C,longax3C=np.zeros(3),np.zeros(3)
strfile='2021-02-04-21_37_42_homogeneoussymmetricity_data'
gaverageseries=data['gaverageseries']
symetaseries=data['symetaseries']
data = np.load(strfile+'.npz')
idxeta,idxgavg=9,9
eigvalseries = data['eigvalseries']
homoeigout=np.mean(np.squeeze(eigvalseries[idxgavg,-1,:,0]))
''' share params '''
gaverage,eta = gaverageseries[idxgavg],symetaseries[idxeta]
''' homo'''
gmat = np.ones((2,2))*gaverage**2/2
eigvgm,eigvecgm=la.eig(gmat)
radius_g = np.sqrt(eigvgm[0])
homolongax = radius_g*(1+eta)
eigout3C[0],longax3C[0]=homoeigout,homolongax
'---------------------------------------'
strfile='2021-02-02-13_57_13_heterrIIsymmetricity_data'
data = np.load(strfile+'.npz')
idxeta,idxgavg=9,9
eigvalseries = data['eigvalseries']
heterrIIeigout=np.mean(np.squeeze(eigvalseries[idxgavg,-1,:,0]))
''' heterogeneous rII ''' 
gmat = np.zeros((2,2))
gmat[1,1]=(np.sqrt(2)*gaverage)**2
eigvgm,eigvecgm=la.eig(gmat)
radius_g = np.sqrt(eigvgm[1])
heterrIIlongax = radius_g*(1+eta)
eigout3C[2],longax3C[2]=heterrIIeigout,heterrIIlongax
'---------------------------------------'
strfile='2021-02-02-17_07_43_heterNotrEEsymmetricity_data'
data = np.load(strfile+'.npz')
idxeta,idxgavg=9,9
eigvalseries = data['eigvalseries']
heterNotrEEeigout=np.mean(np.squeeze(eigvalseries[idxgavg,-1,:,0]))
''' heterogeneous NotrEE '''
gmat = np.ones((2,2))*(np.sqrt(2.0/3.0)*gaverage)**2
gmat[0,0]=0
eigvgm,eigvecgm=la.eig(gmat)
radius_g = np.sqrt(eigvgm[1])
heterNotrEElongax = radius_g*(1+eta)
eigout3C[1],longax3C[1]=heterNotrEEeigout,heterNotrEElongax
'---------------------------------------'
print('eigenoutliers')
plt.plot((eigout3C+2.5)/2.5,longax3C)#longax3C)#eigout3C)#,longax3C)
plt.show()

eigenoutliers


In [132]:
# strfile='2021-02-04-21_37_42_homogeneoussymmetricity_data'
# strfile='2021-02-02-17_07_43_heterNotrEEsymmetricity_data'
strfile='2021-02-02-13_57_13_heterrIIsymmetricity_data'
data = np.load(strfile+'.npz')
# fig,ax=plt.subplots(figsize=(5,5))
eigvalseries = data['eigvalseries']
# perturbed
mvecseries,nvecseries=data['mvecseries'],data['nvecseries']
eigvAm=data['eigvAm']
gaverageseries=data['gaverageseries']
symetaseries=data['symetaseries']
''' how eigenvalue outlier changes with g_bar '''
# three eta 0.2,0.5,0.8
# np.zeros((ngavg,neta,ntrial,nrank))
ngavg,neta,ntrial,N,nrank=np.shape(mvecseries)
thetamaxseries=np.zeros((ngavg,neta,ntrial,2,2))
scale_std=3.0
for idxgavg,gaverage in enumerate(gaverageseries):
    for idxeta,eta in enumerate(symetaseries):       
        for ktrial in range(ntrial):
            meanmE,meanmI,meannE,meannI=np.zeros(nrank),np.zeros(nrank),np.zeros(nrank),np.zeros(nrank)
            meigvecloading,neigvecloading=mvecseries[idxgavg,idxeta,ktrial,:,0].copy(),nvecseries[idxgavg,idxeta,ktrial,:,0].copy()
            meigvecloading,neigvecloading=np.reshape(meigvecloading,(2*NE,1)),np.reshape(neigvecloading,(2*NE,1))
            # m vector
            # puring E
            mEmat = np.squeeze(meigvecloading[:NE,irank])
            meanmEtotal,stdmEtotal = np.mean(mEmat),np.std(mEmat)
            varmE = mEmat - meanmEtotal
            idxwhere = np.where(np.abs(varmE)>scale_std*stdmEtotal)
            mEmat[idxwhere]=meanmEtotal
            meigvecloading[:NE,irank]=mEmat
            meanmE[irank]=np.mean(mEmat)

            # puring I
            mImat = np.squeeze(meigvecloading[NE:,irank])
            meanmItotal,stdmItotal = np.mean(mImat),np.std(mImat)
            varmI = mImat - meanmItotal
            idxwhere = np.where(np.abs(varmI)>scale_std*stdmItotal)
            mImat[idxwhere]=meanmItotal
            meigvecloading[NE:,irank]=mImat
            meanmI[irank]=np.mean(mImat)
            
            # n vector
            # puring E
            nEmat = np.squeeze(neigvecloading[:NE,irank])
            meannEtotal,stdnEtotal = np.mean(nEmat),np.std(nEmat)
            varnE = nEmat - meannEtotal
            idxwhere = np.where(np.abs(varnE)>scale_std*stdnEtotal)
            nEmat[idxwhere]=meannEtotal
            neigvecloading[:NE,irank]=nEmat
            meannE[irank]=np.mean(nEmat)

            # puring I
            nImat = np.squeeze(neigvecloading[NE:,irank])
            meannItotal,stdnItotal = np.mean(nImat),np.std(nImat)
            varnI = nImat - meannItotal
            idxwhere = np.where(np.abs(varnI)>scale_std*stdnItotal)
            nImat[idxwhere]=meannItotal
            neigvecloading[NE:,irank]=nImat
            meannI[irank]=np.mean(nImat)

            ''' calculate directions '''
            noiseE,noiseI = np.zeros((NE,2,nrank)),np.zeros((NE,2,nrank))
            dirvecE,dirvecI=np.zeros((2,2)),np.zeros((2,2))
            for irank in range(nrank):
                # E 0 M 1 N
                noiseE[:,0,irank],noiseE[:,1,irank]= np.squeeze(neigvecloading[:NE,irank])-meannE[irank],np.squeeze(meigvecloading[:NE,irank])-meanmE[irank]
                noiseI[:,0,irank],noiseI[:,1,irank]= np.squeeze(neigvecloading[NE:,irank])-meannI[irank],np.squeeze(meigvecloading[NE:,irank])-meanmI[irank]
            # m1n1
            covdirE,covdirI=np.squeeze(noiseE[:,:,0]).T@np.squeeze(noiseE[:,:,0]),np.squeeze(noiseI[:,:,0]).T@np.squeeze(noiseI[:,:,0])
            _,dirvecE=la.eig(covdirE)
            thetamaxseries[idxgavg,idxeta,ktrial,0,0],thetamaxseries[idxgavg,idxeta,ktrial,0,1]  =np.arctan2(dirvecE[1,0],dirvecE[0,0]),np.arctan2(dirvecE[1,1],dirvecE[0,1])
            _,dirvecI=la.eig(covdirI)  
            thetamaxseries[idxgavg,idxeta,ktrial,1,0],thetamaxseries[idxgavg,idxeta,ktrial,1,1]  =np.arctan2(dirvecI[1,0],dirvecI[0,0]),np.arctan2(dirvecI[1,1],dirvecI[0,1])

idxgavg=9
abstheta = np.abs(thetamaxseries[idxgavg,:,:,1,0])
ax.plot(np.mean(np.squeeze(eigvalseries[idxgavg,:,:,0]),axis=1),np.mean(-abstheta,axis=1),color='b',linestyle='-',linewidth=1.5)

In [126]:
idxgavg=2
# fig,ax=plt.subplots(figsize=(5,5))
abstheta = np.abs(thetamaxseries[idxgavg,:,:,0,0])
ax.plot(np.mean(np.squeeze(eigvalseries[idxgavg,:,:,0]),axis=1),np.mean(-abstheta,axis=1),color='r',linestyle='-',linewidth=1.5)

## RANDOM $g_{\rm{mat}}$, and sorting according to its effect random gain $r_g$

In [ ]:
''' #### saving data structure
now = time.strftime("%Y-%m-%d-%H_%M_%S",time.localtime(time.time()))

Jparameters = np.zeros(4+1)
Jparameters[0],Jparameters[1],Jparameters[2],Jparameters[3]=JEE,JEI,JIE,JII
Jparameters[4]=NE

gparameters=np.zeros(4+2+1)
gparameters[:4],gparameters[4],gparameters[5],gparameters[6]=grand,gaverage,r_g,eta

np.savez(now+'_hetersymmetricity_data',Jparameters=Jparameters,gparameters=gparameters,mvecseries=mvecseries,nvecseries=nvecseries,eigvalseries=eigvalseries,Reigvecseries=Reigvecseries,Leigvecseries=Leigvecseries,meigvecAm=meigvecAm,neigvecAm=neigvecAm,eigvAm=eigvAm,thetamaxseries=thetamaxseries)
'''

In [316]:
# strfile='2021-02-04-21_37_42_homogeneoussymmetricity_data'
strfile='2021-02-02-17_07_43_heterNotrEEsymmetricity_data'
data = np.load(strfile+'.npz')
fig,ax=plt.subplots(figsize=(5,5))
eigvalseries = data['eigvalseries']
# perturbed
mvecseries,nvecseries=data['mvecseries'],data['nvecseries']
eigvAm=data['eigvAm']
gaverageseries=data['gaverageseries']
symetaseries=data['symetaseries']
ngavg,neta,ntrial,N,nrank=np.shape(mvecseries)
# neta,ntrial,N,nrank=np.shape(mvecseries)
# properties
thetamaxseries=np.zeros((neta,ntrial,2,2))
# random 
gaverage=1.0
while(1):
    grand = gmatamplitude_eig(gaverage**2,2)
    idxeff = np.where(grand>0)
    if (np.shape(idxeff)[1]>3):
        np.random.shuffle(grand)
        break
print('random amp:',grand)
''' test r_g and gbar'''
# theoretical M (grandom) matrix 
# gparameters=data['gparameters']
# grand=gparameters[:4]
# xee,xei,xie,xii=grand[0],grand[1],grand[2],grand[3]
grand=np.ones(4)*gaverage/np.sqrt(2)
xee,xei,xie,xii=grand[0],grand[1],grand[2],grand[3]
gmat = np.array([[xee**2,xei**2],[xie**2,xii**2]])
# first do not multiply at
gaverage_=0
for i in range(2):
    for j in range(2):
        gaverage_+=gmat[i,j]/2 # ntype=2
gaverage_=np.sqrt(gaverage_)
eigvgm,eigvecgm=la.eig(gmat) 
r_g2=np.max(eigvgm)
r_g = np.sqrt(r_g2)
print('gbar:,',gaverage_,'; effective gain:',r_g)
idxgavg=9
for idxeta, eta in enumerate(symetaseries):
    for ktrial in range(ntrial):
        meanmE,meanmI,meannE,meannI=np.zeros(nrank),np.zeros(nrank),np.zeros(nrank),np.zeros(nrank)
        # meigvecloading,neigvecloading=mvecseries[idxeta,ktrial,:,i].copy(),nvecseries[idxeta,ktrial,:,i].copy()
        meigvecloading,neigvecloading=mvecseries[idxgavg,idxeta,ktrial,:,0].copy(),nvecseries[idxgavg,idxeta,ktrial,:,0].copy()
        meigvecloading,neigvecloading=np.reshape(meigvecloading,(2*NE,1)),np.reshape(neigvecloading,(2*NE,1))
        scale_std=3.0
        for irank in range(nrank):
            # m vector
            # puring E
            mEmat = np.squeeze(meigvecloading[:NE,irank])
            meanmEtotal,stdmEtotal = np.mean(mEmat),np.std(mEmat)
            varmE = mEmat - meanmEtotal
            idxwhere = np.where(np.abs(varmE)>scale_std*stdmEtotal)
            mEmat[idxwhere]=meanmEtotal
            meigvecloading[:NE,irank]=mEmat
            meanmE[irank]=np.mean(mEmat)

            # puring I
            mImat = np.squeeze(meigvecloading[NE:,irank])
            meanmItotal,stdmItotal = np.mean(mImat),np.std(mImat)
            varmI = mImat - meanmItotal
            idxwhere = np.where(np.abs(varmI)>scale_std*stdmItotal)
            mImat[idxwhere]=meanmItotal
            meigvecloading[NE:,irank]=mImat
            meanmI[irank]=np.mean(mImat)
            
            # n vector
            # puring E
            nEmat = np.squeeze(neigvecloading[:NE,irank])
            meannEtotal,stdnEtotal = np.mean(nEmat),np.std(nEmat)
            varnE = nEmat - meannEtotal
            idxwhere = np.where(np.abs(varnE)>scale_std*stdnEtotal)
            nEmat[idxwhere]=meannEtotal
            neigvecloading[:NE,irank]=nEmat
            meannE[irank]=np.mean(nEmat)

            # puring I
            nImat = np.squeeze(neigvecloading[NE:,irank])
            meannItotal,stdnItotal = np.mean(nImat),np.std(nImat)
            varnI = nImat - meannItotal
            idxwhere = np.where(np.abs(varnI)>scale_std*stdnItotal)
            nImat[idxwhere]=meannItotal
            neigvecloading[NE:,irank]=nImat
            meannI[irank]=np.mean(nImat)

        ''' calculate directions '''
        noiseE,noiseI = np.zeros((NE,2,nrank)),np.zeros((NE,2,nrank))
        dirvecE,dirvecI=np.zeros((2,2)),np.zeros((2,2))
        for irank in range(nrank):
            # E 0 M 1 N
            noiseE[:,0,irank],noiseE[:,1,irank]= np.squeeze(neigvecloading[:NE,irank])-meannE[irank],np.squeeze(meigvecloading[:NE,irank])-meanmE[irank]
            noiseI[:,0,irank],noiseI[:,1,irank]= np.squeeze(neigvecloading[NE:,irank])-meannI[irank],np.squeeze(meigvecloading[NE:,irank])-meanmI[irank]
        # m1n1
        covdirE,covdirI=np.squeeze(noiseE[:,:,0]).T@np.squeeze(noiseE[:,:,0]),np.squeeze(noiseI[:,:,0]).T@np.squeeze(noiseI[:,:,0])
        _,dirvecE=la.eig(covdirE)
        thetamaxseries[idxeta,ktrial,0,0],thetamaxseries[idxeta,ktrial,0,1]  =np.arctan2(dirvecE[1,0],dirvecE[0,0]),np.arctan2(dirvecE[1,1],dirvecE[0,1])
        _,dirvecI=la.eig(covdirI)  
        thetamaxseries[idxeta,ktrial,1,0],thetamaxseries[idxeta,ktrial,1,1]  =np.arctan2(dirvecI[1,0],dirvecI[0,0]),np.arctan2(dirvecI[1,1],dirvecI[0,1])   
            


random amp: [0.35930778 0.65044661 1.0327996  0.61736709]
gbar:, 0.9999999999999999 ; effective gain: 0.9999999999999999


In [146]:
# fig,ax=plt.subplots(figsize=(12,3))  
ax.plot(np.mean(np.squeeze(eigvalseries[-1,:,:,0]),axis=1),np.mean(np.squeeze(thetamaxseries[:,:,0,0]),axis=1),color='red',alpha=0.5)
ax.plot(np.mean(np.squeeze(eigvalseries[-1,:,:,0]),axis=1),np.mean(np.squeeze(thetamaxseries[:,:,1,0]),axis=1),color='blue',alpha=0.5)

## heterogeneous degree of symmetry
* eigenvalue outliers, spectrum
* OVERLAP LOADING SPACE

In [317]:
''' ----------- eigenvalue outliers, eigenspectrum --------------- '''

xticks = np.linspace(-4.0,3.0,3)
xlims = [-4.1,3.1]
yticks = np.linspace(-1.3,1.3,3)
ylims = [-1.31,1.31]
theta = np.linspace(0, 2 * np.pi, 200)
idxxx=0
# strfile='2021-02-07-13_00_51_hetersymmetricityBlockrEE_data'
# strfile='2021-02-07-14_21_01_hetersymmetricityBlockrII_data'
# strfile='2021-02-07-17_41_32_hetersymmetricityBlockrII_cm_data'
# strfile='2021-02-07-18_00_24_hetersymmetricityBlockrII_cm_rdrg_data'
# strfile='2021-02-07-15_42_26_hetersymmetricityBlockNotrEE_data'
strfile='2021-02-09-21_15_14_hetersymmetricityBlockNotrEE_data'
fig,ax=plt.subplots(3,3,figsize=(16,6))
data = np.load(strfile+'.npz')
nrank=1
# # regenerate unperturbed low rank
# Jparameters=data['Jparameters']
# NE = int(Jparameters[4])
# P = np.zeros((2,2))
# P = np.array([[Jparameters[0],-Jparameters[1]],[Jparameters[2],-Jparameters[3]]])
# Am=np.zeros((NE*2,NE*2))
# Am[:NE,:NE],Am[:NE,NE:]=P[0,0]/NE,P[0,1]/NE
# Am[NE:,:NE],Am[NE:,NE:]=P[1,0]/NE,P[1,1]/NE
# eigvAm,eigveclAm,eigvecrAm=scpla.eig(Am,left=True,right=True)

# perturbed
symetaseries = np.linspace(0.0,1.0,11) 
mvecseries,nvecseries=data['mvecseries'],data['nvecseries']
eigvalseries=data['eigvalseries']
eigvAm=data['eigvAm']

idxxsymeta = np.array([0,4,9]) # a series of symmetricity value
nrank,ntrial,neta=1,1,len(symetaseries)
gparameters=data['gparameters']
# random gain and symmetricity
gaverage=gparameters[4]#1.0
grand=gparameters[:4]
print('grand:',grand)
''' test r_g and gbar'''
# theoretical M (grandom) matrix 
xee,xei,xie,xii=grand[0],grand[1],grand[2],grand[3]
gmat = np.array([[xee**2,xei**2],[xie**2,xii**2]])
# first do not multiply at
gaverage_=0
for i in range(2):
    for j in range(2):
        gaverage_+=gmat[i,j]/2 # ntype=2
gaverage_=np.sqrt(gaverage_)
eigvgm,eigvecgm=la.eig(gmat) 
r_g2=np.max(eigvgm)
r_g = np.sqrt(r_g2)
print('gbar:,',gaverage_,'; effective gain:',r_g)
''' C0 rules '''
xax=np.arange(NE*2)
XX,YY=np.meshgrid(xax,xax)
idxdowntriangle=np.where(XX>YY)
idxuptriangle=np.where(XX<YY)

for didxeta, idxeta in enumerate(idxxsymeta):
    eta=symetaseries[idxeta]
    Xsym=np.random.randn(NE*2,NE*2)/np.sqrt(NE)
    XsymT=Xsym.copy().T
    Yidendown=np.random.randn(NE*2,NE*2)/np.sqrt(NE)
    Yidenup=np.random.randn(NE*2,NE*2)/np.sqrt(NE)
    Xinit = Xsym.copy()
    Xinit[idxdowntriangle]=Xsym[idxdowntriangle]*np.sqrt(eta)+Yidendown[idxdowntriangle].copy()*np.sqrt(1-eta)
    Xinit[idxuptriangle]=XsymT[idxuptriangle]*np.sqrt(eta)+Yidenup[idxuptriangle].copy()*np.sqrt(1-eta)
    Yind=np.random.randn(NE*2,NE*2)/np.sqrt(NE)
    # Xinit=Xinit.copy()/np.sqrt(2)
    # ''' rII '''
    # X=Yind.copy()
    # # X[:NE,:NE]=Xinit[:NE,:NE].copy()
    # X[NE:,NE:]=Xinit[NE:,NE:].copy()
    ''' NotrEE '''
    X=Xinit.copy()
    # X[:NE,:NE]=Xinit[:NE,:NE].copy()
    X[:NE,:NE]=Yind[:NE,:NE].copy()

    # ''' homogeneous '''
    X*=(gaverage/np.sqrt(2))
    # # ''' heterogeneous rII '''
    # X[:NE,:NE]=0
    # X[:NE,NE:]=0
    # X[NE:,:NE]=0
    # X[NE:,NE:]*=np.sqrt(2)*gaverage
    # # ''' heterogeneous not rEE '''
    # X[:NE,:NE]=0
    # X[:NE,NE:]*=(np.sqrt(2.0/3.0)*gaverage)
    # X[NE:,:NE]*=(np.sqrt(2.0/3.0)*gaverage)
    # X[NE:,NE:]*=(np.sqrt(2.0/3.0)*gaverage)
    # # random 
    # X[:NE,:NE]*=grand[0]
    # X[:NE,NE:]*=grand[1]
    # X[NE:,:NE]*=grand[2]
    # X[NE:,NE:]*=grand[3]
    # theogrand = (grand[1]*grand[2]*2+grand[0]**2+grand[3]**2)/2
    symmetricityX=np.sum(np.sum(X.T*X))/(NE*2)
    print(symmetricityX,eta)
    ''' longshort axis '''
    print(r_g)
    longaxis,shortaxis=(1+eta)*r_g ,(1-eta)*r_g 
    xr = longaxis*np.cos(theta)
    yr = shortaxis*np.sin(theta)
    symmetricityX=np.sum(np.sum(X.T*X))/(NE*2)
    print(symmetricityX,eta)
    # overall
    J = X.copy()+Am.copy()
    # properties
    eigvJ,eigrvecJ=la.eig(J)
    inveigrvecJ=la.inv(eigrvecJ)
    
    real_eigsbulk,imag_eigsbulk=np.real(eigvJ[nrank:]),np.imag(eigvJ[nrank:])
    ax[didxeta][idxxx].scatter(real_eigsbulk,imag_eigsbulk,s=3,color='blue',alpha=0.25)
    ax[didxeta][idxxx].scatter(np.real(eigvalseries[idxeta,:,0]),np.imag(eigvalseries[idxeta,:,0]),s=5,color='blue',alpha=0.5)
    ax[didxeta][idxxx].scatter(np.real(eigvJ[:nrank]),np.imag(eigvJ[:nrank]),marker='^',s=20,color='red',alpha=1.0)
    # xa = gaverage*np.cos(theta)
    # ya = gaverage*np.sin(theta)
    # ax[0].plot(xa, ya, color="darkred", linewidth=1.5,label=r'$r_g$')
    ax[didxeta][idxxx].plot(xr, yr, color="darkred", linewidth=2.0,linestyle='--',label=r'ellipse')
    ax[didxeta][idxxx].set_xlim(xlims)
    ax[didxeta][idxxx].set_ylim(ylims)
    ax[didxeta][idxxx].set_xticks(xticks)
    ax[didxeta][idxxx].set_yticks(yticks)
    ax[didxeta][idxxx].set_aspect('equal')
    ax[didxeta][idxxx].legend()
    ax[didxeta][idxxx].grid()
    ax[didxeta][idxxx].set_aspect('equal')

for didxeta in range(3):
    ax[didxeta][idxxx].set_xlim(xlims)
    ax[didxeta][idxxx].set_ylim(ylims)
    ax[didxeta][idxxx].set_xticks([])
    ax[didxeta][idxxx].set_yticks([])
    if ax[didxeta][idxxx].is_last_row():
        ax[didxeta][idxxx].set_xticks(xticks)
        ax[didxeta][idxxx].set_xlabel(r'$Re(\lambda)$',fontsize=12)
    if ax[didxeta][idxxx].is_first_col():
        ax[didxeta][idxxx].set_yticks(yticks)
        ax[didxeta][idxxx].set_ylabel(r'$Im(\lambda}$',fontsize=12)
plt.tight_layout()


grand: [0.70710678 0.70710678 0.70710678 0.70710678]
gbar:, 0.9999999999999999 ; effective gain: 0.9999999999999999
0.0005778416160932575 0.0
0.9999999999999999
0.0005778416160932575 0.0
0.30017798636429455 0.4
0.9999999999999999
0.30017798636429455 0.4
0.6767952937432496 0.9
0.9999999999999999
0.6767952937432496 0.9


In [302]:
xticks = np.linspace(-0.3,0.4,2)
xlims = [-0.31,0.41]
yticks = np.linspace(-0.1,0.1,3)
ylims = [-0.11,0.11]
''' ------------------- loading space -------------------- '''
# figs2,axs2=plt.subplots(figsize=(5,5))
eigvalseries = data['eigvalseries']
# perturbed
mvecseries,nvecseries=data['mvecseries'],data['nvecseries']
eigvAm=data['eigvAm']
''' how eigenvalue outlier changes with g_bar '''
# three eta 0.2,0.5,0.8
# np.zeros((ngavg,neta,ntrial,nrank))
neta,ntrial,N,nrank=np.shape(mvecseries)
thetamaxseries=data['thetamaxseries']
scale_std=3.0
for didxeta, idxeta in enumerate(idxxsymeta):   
    ktrial=6
    meanmE,meanmI,meannE,meannI=np.zeros(nrank),np.zeros(nrank),np.zeros(nrank),np.zeros(nrank)
    meigvecloading,neigvecloading=mvecseries[idxeta,ktrial,:,0].copy(),nvecseries[idxeta,ktrial,:,0].copy()
    if didxeta==2:
        meigvecloading,neigvecloading=-meigvecloading,-neigvecloading
    meigvecloading,neigvecloading=np.reshape(meigvecloading,(2*NE,1)),np.reshape(neigvecloading,(2*NE,1))
    # m vector
    # puring E
    mEmat = np.squeeze(meigvecloading[:NE,0])
    meanmEtotal,stdmEtotal = np.mean(mEmat),np.std(mEmat)
    varmE = mEmat - meanmEtotal
    idxwhere = np.where(np.abs(varmE)>scale_std*stdmEtotal)
    mEmat[idxwhere]=meanmEtotal
    meigvecloading[:NE,0]=mEmat
    meanmE[0]=np.mean(mEmat)

    # puring I
    mImat = np.squeeze(meigvecloading[NE:,0])
    meanmItotal,stdmItotal = np.mean(mImat),np.std(mImat)
    varmI = mImat - meanmItotal
    idxwhere = np.where(np.abs(varmI)>scale_std*stdmItotal)
    mImat[idxwhere]=meanmItotal
    meigvecloading[NE:,0]=mImat
    meanmI[0]=np.mean(mImat)
    
    # n vector
    # puring E
    nEmat = np.squeeze(neigvecloading[:NE,0])
    meannEtotal,stdnEtotal = np.mean(nEmat),np.std(nEmat)
    varnE = nEmat - meannEtotal
    idxwhere = np.where(np.abs(varnE)>scale_std*stdnEtotal)
    nEmat[idxwhere]=meannEtotal
    neigvecloading[:NE,0]=nEmat
    meannE[0]=np.mean(nEmat)

    # puring I
    nImat = np.squeeze(neigvecloading[NE:,0])
    meannItotal,stdnItotal = np.mean(nImat),np.std(nImat)
    varnI = nImat - meannItotal
    idxwhere = np.where(np.abs(varnI)>scale_std*stdnItotal)
    nImat[idxwhere]=meannItotal
    neigvecloading[NE:,0]=nImat
    meannI[0]=np.mean(nImat)

    ''' calculate directions '''
    noiseE,noiseI = np.zeros((NE,2,nrank)),np.zeros((NE,2,nrank))
    dirvecE,dirvecI=np.zeros((2,2)),np.zeros((2,2))
    for irank in range(nrank):
        # E 0 M 1 N
        noiseE[:,0,irank],noiseE[:,1,irank]= np.squeeze(neigvecloading[:NE,irank])-meannE[irank],np.squeeze(meigvecloading[:NE,irank])-meanmE[irank]
        noiseI[:,0,irank],noiseI[:,1,irank]= np.squeeze(neigvecloading[NE:,irank])-meannI[irank],np.squeeze(meigvecloading[NE:,irank])-meanmI[irank]
    # m1n1
    covdirE,covdirI=np.squeeze(noiseE[:,:,0]).T@np.squeeze(noiseE[:,:,0]),np.squeeze(noiseI[:,:,0]).T@np.squeeze(noiseI[:,:,0])
    _,dirvecE=la.eig(covdirE)
    # thetamaxseries[idxgavg,idxeta,ktrial,0,0],thetamaxseries[idxgavg,idxeta,ktrial,0,1]  =np.arctan2(dirvecE[1,0],dirvecE[0,0]),np.arctan2(dirvecE[1,1],dirvecE[0,1])
    _,dirvecI=la.eig(covdirI)  
    # thetamaxseries[idxgavg,idxeta,ktrial,1,0],thetamaxseries[idxgavg,idxeta,ktrial,1,1]  =np.arctan2(dirvecI[1,0],dirvecI[0,0]),np.arctan2(dirvecI[1,1],dirvecI[0,1])

    fig,ax=plt.subplots(figsize=(5,3))
    ax.scatter(neigvecloading[NE:,0],meigvecloading[NE:,0],s=5.0,c='blue',alpha=0.5)
    ax.scatter(neigvecloading[:NE,0],meigvecloading[:NE,0],s=5.0,c='red',alpha=0.5)

    ax.plot([meannE[0],meannE[0]+dirvecE[0,0]],[meanmE[0],meanmE[0]+dirvecE[1,0]],color='r',linestyle='--',linewidth=1.5)
    ax.plot([meannE[0],meannE[0]+dirvecE[0,1]],[meanmE[0],meanmE[0]+dirvecE[1,1]],color='r',linestyle='--',linewidth=1.5)
    ax.plot([meannI[0],meannI[0]+dirvecI[0,0]],[meanmI[0],meanmI[0]+dirvecI[1,0]],color='b',linestyle='--',linewidth=1.5)
    ax.plot([meannI[0],meannI[0]+dirvecI[0,1]],[meanmI[0],meanmI[0]+dirvecI[1,1]],color='b',linestyle='--',linewidth=1.5)

    mE,nE=meigvecloading[:NE,0].copy(),neigvecloading[:NE,0].copy()
    confidence_ellipse(np.real(nE),np.real(mE),ax,edgecolor='orange')
    ax.scatter(nE,mE,s=10,c='red',alpha=0.5)
    mI,nI=meigvecloading[NE:,0].copy(),neigvecloading[NE:,0].copy()
    confidence_ellipse(np.real(nI),np.real(mI),ax,edgecolor='cyan')
    ax.scatter(nI,mI,s=10,c='blue',alpha=0.5)
    ax.axvline(c='grey', lw=1)
    ax.axhline(c='grey', lw=1)
    xxx=np.linspace(0.0,1.0,10)
    yyy=np.tan(thetamaxseries[idxeta,ktrial,0,0])*xxx
    # ax.plot(xxx,yyy,color='green',linestyle='--',linewidth=2.0)
    ax.set_xlim(xlims)
    ax.set_ylim(ylims)
    ax.set_xticks(xticks)
    ax.set_yticks(yticks)
    ax.set_aspect('equal')


(2, 2)
(2, 2)
(2, 2)
(2, 2)
(2, 2)
(2, 2)


In [ ]:
ktrial=6
abstheta = np.abs(thetamaxseries[idxgavg,:,:,1,0])
ax.plot(np.mean(np.squeeze(eigvalseries[idxgavg,:,:,0]),axis=1),np.mean(-abstheta,axis=1),color='b',linestyle='-',linewidth=1.5)

In [314]:
xticks = np.linspace(-4.0,3.0,5)
xlims = [-4.1,3.1]
yticks = np.linspace(-1.3,1.3,3)
ylims = [-1.31,1.31]
theta = np.linspace(0, 2 * np.pi, 200)
idxxx=0
# strfile='2021-02-07-13_00_51_hetersymmetricityBlockrEE_data'
# strfile='2021-02-07-14_21_01_hetersymmetricityBlockrII_data'
# strfile='2021-02-07-15_42_26_hetersymmetricityBlockNotrEE_data'
strfile='2021-02-09-21_15_14_hetersymmetricityBlockNotrEE_data'

data = np.load(strfile+'.npz')#'2021-01-19-12_02_53_heterogeneousgII_cloudsdata.npz')
nrank=1
# regenerate unperturbed low rank
Jparameters=data['Jparameters']
NE = int(Jparameters[4])

# perturbed
mvecseries,nvecseries=data['mvecseries'],data['nvecseries']
eigvalseries=data['eigvalseries']
eigvAm=data['eigvAm']
# symetaseries=data['symetaseries']
symetaseries = np.linspace(0.0,1.0,11)
''' how eigenvalue outlier changes with g_bar '''
# three eta 0.2,0.5,0.8
# np.zeros((ngavg,neta,ntrial,nrank))
neta,ntrial,N,nrank=np.shape(mvecseries)
NE=int(N/2)

homopearsoncorr_rho=np.zeros((neta,ntrial,2,nrank))
homothetamaxseries=np.zeros((neta,ntrial,2,2))
homoeigtotalseries=np.zeros((neta,ntrial,nrank))

''' get eigenvalue outlier series  '''
homoeigtotalseries[:,:,:]=eigvalseries
for idxeta, eta in enumerate(symetaseries):
    for ktrial in range(ntrial):
        meanmE,meanmI,meannE,meannI=np.zeros(nrank),np.zeros(nrank),np.zeros(nrank),np.zeros(nrank)
        # print(jdxgavg)
        meigvecloading,neigvecloading=mvecseries[idxeta,ktrial,:,0].copy(),nvecseries[idxeta,ktrial,:,0].copy()
        meigvecloading,neigvecloading=np.reshape(meigvecloading,(2*NE,1)),np.reshape(neigvecloading,(2*NE,1))
        scale_std=3.0
        for irank in range(nrank):
            # m vector
            # puring E
            mEmat = np.squeeze(meigvecloading[:NE,irank])
            meanmEtotal,stdmEtotal = np.mean(mEmat),np.std(mEmat)
            varmE = mEmat - meanmEtotal
            idxwhere = np.where(np.abs(varmE)>scale_std*stdmEtotal)
            mEmat[idxwhere]=meanmEtotal
            meigvecloading[:NE,irank]=mEmat
            meanmE[irank]=np.mean(mEmat)

            # puring I
            mImat = np.squeeze(meigvecloading[NE:,irank])
            meanmItotal,stdmItotal = np.mean(mImat),np.std(mImat)
            varmI = mImat - meanmItotal
            idxwhere = np.where(np.abs(varmI)>scale_std*stdmItotal)
            mImat[idxwhere]=meanmItotal
            meigvecloading[NE:,irank]=mImat
            meanmI[irank]=np.mean(mImat)
            
            # n vector
            # puring E
            nEmat = np.squeeze(neigvecloading[:NE,irank])
            meannEtotal,stdnEtotal = np.mean(nEmat),np.std(nEmat)
            varnE = nEmat - meannEtotal
            idxwhere = np.where(np.abs(varnE)>scale_std*stdnEtotal)
            nEmat[idxwhere]=meannEtotal
            neigvecloading[:NE,irank]=nEmat
            meannE[irank]=np.mean(nEmat)

            # puring I
            nImat = np.squeeze(neigvecloading[NE:,irank])
            meannItotal,stdnItotal = np.mean(nImat),np.std(nImat)
            varnI = nImat - meannItotal
            idxwhere = np.where(np.abs(varnI)>scale_std*stdnItotal)
            nImat[idxwhere]=meannItotal
            neigvecloading[NE:,irank]=nImat
            meannI[irank]=np.mean(nImat)

        ''' calculate directions '''
        noiseE,noiseI = np.zeros((NE,2,nrank)),np.zeros((NE,2,nrank))
        dirvecE,dirvecI=np.zeros((2,2)),np.zeros((2,2))
        for irank in range(nrank):
            # E 0 M 1 N
            noiseE[:,0,irank],noiseE[:,1,irank]= np.squeeze(neigvecloading[:NE,irank])-meannE[irank],np.squeeze(meigvecloading[:NE,irank])-meanmE[irank]
            noiseI[:,0,irank],noiseI[:,1,irank]= np.squeeze(neigvecloading[NE:,irank])-meannI[irank],np.squeeze(meigvecloading[NE:,irank])-meanmI[irank]
        # m1n1
        covdirE,covdirI=np.squeeze(noiseE[:,:,0]).T@np.squeeze(noiseE[:,:,0]),np.squeeze(noiseI[:,:,0]).T@np.squeeze(noiseI[:,:,0])
        _,dirvecE=la.eig(covdirE)
        homothetamaxseries[idxeta,ktrial,0,0],homothetamaxseries[idxeta,ktrial,0,1] =np.arctan2(dirvecE[1,0],dirvecE[0,0]),np.arctan2(dirvecE[1,1],dirvecE[0,1])
        _,dirvecI=la.eig(covdirI)  
        homothetamaxseries[idxeta,ktrial,1,0],homothetamaxseries[idxeta,ktrial,1,1]  =np.arctan2(dirvecI[1,0],dirvecI[0,0]),np.arctan2(dirvecI[1,1],dirvecI[0,1])
        ## pearson correlation 
        normmE,normmI,normnE,normnI=np.reshape(noiseE[:,0,0],(NE,1)),np.reshape(noiseI[:,0,0],(NE,1)),np.reshape(noiseE[:,1,0],(NE,1)),np.reshape(noiseI[:,1,0],(NE,1))
        normmE,normmI,normnE,normnI=normmE/la.norm(normmE),normmI/la.norm(normmI),normnE/la.norm(normnE),normnI/la.norm(normnI)
        homopearsoncorr_rho[idxeta,ktrial,0,0],homopearsoncorr_rho[idxeta,ktrial,1,0]=normmE.T@normnE,normmI.T@normnI

    
    # fig,ax=plt.subplots(figsize=(4,4))  
    # ax.scatter(neigvecloading[NE:,0],meigvecloading[NE:,0],s=5.0,c='blue',alpha=0.5)
    # ax.scatter(neigvecloading[:NE,0],meigvecloading[:NE,0],s=5.0,c='red',alpha=0.5)

    # # ax.plot([meannE[0],meannE[0]+dirvecE[0,0]],[meanmE[0],meanmE[0]+dirvecE[1,0]],color='r',linestyle='--',linewidth=1.5)
    # # ax.plot([meannE[0],meannE[0]+dirvecE[0,1]],[meanmE[0],meanmE[0]+dirvecE[1,1]],color='r',linestyle='--',linewidth=1.5)
    # # ax.plot([meannI[0],meannI[0]+dirvecI[0,0]],[meanmI[0],meanmI[0]+dirvecI[1,0]],color='b',linestyle='--',linewidth=1.5)
    # # ax.plot([meannI[0],meannI[0]+dirvecI[0,1]],[meanmI[0],meanmI[0]+dirvecI[1,1]],color='b',linestyle='--',linewidth=1.5)

    # mE,nE=meigvecloading[:NE,0].copy(),neigvecloading[:NE,0].copy()
    # confidence_ellipse(np.real(nE),np.real(mE),ax,edgecolor='orange')
    # ax.scatter(nE,mE,s=10,c='red',alpha=0.5)
    # mI,nI=meigvecloading[NE:,0].copy(),neigvecloading[NE:,0].copy()
    # confidence_ellipse(np.real(nI),np.real(mI),ax,edgecolor='cyan')
    # ax.scatter(nI,mI,s=10,c='blue',alpha=0.5)
    # ax.axvline(c='grey', lw=1)
    # ax.axhline(c='grey', lw=1)
    # # xxx=np.linspace(0.0,1.0,10)
    # # yyy=np.tan(thetamaxseries[iddg,jdxgavg,idxeta,6,0,0])*xxx
    # # ax.plot(xxx,yyy,color='green',linestyle='--',linewidth=2.0)
    # # ax.set_xlim(xlims)
    # # ax.set_ylim(ylims)
    # # ax.set_xticks(xticks)
    # # ax.set_yticks(yticks)
    # # ax.set_aspect('equal')
            


In [315]:
import mpl_toolkits.axisartist as axisartist
# #object ax
''' font setting '''
font_format = {'family':'Calibre', 'size':16}#:'Times New Roman', 'size':34}
# limit 
xticks = np.linspace(0.0,1.0,2)
xlims = [-.01,1.01]
yticks = np.linspace(-1.0,0.0,2)
ylims = [-1.01,0.01]
delstr='bottom'
signeigv=-1

# xticks = np.linspace(0.0,1.0,2)
# xlims = [-.01,1.01]
# yticks = np.linspace(0.0,1.0,2)
# ylims = [-.01,1.01]
# delstr='top'
# sigeigv=1

fig,ax=plt.subplots(1,2,figsize=(9,5))
i=0
eigtemp=np.zeros((neta+1,1))
pearsoncorrtemp=np.zeros((neta+1,2))
eigtemp[0,0]=eigvAm[0]
eigtemp[1:,0]=np.mean(np.squeeze(homoeigtotalseries[:,:,0]),axis=1)
pearsoncorrtemp[1:,0]=np.mean(np.squeeze(homopearsoncorr_rho[:,:,0,0]),axis=1)
pearsoncorrtemp[1:,1]=np.mean(np.squeeze(homopearsoncorr_rho[:,:,1,0]),axis=1)
symetaseries_=np.zeros(neta+1)
symetaseries_[1:]=symetaseries.copy()
ax[0].plot(symetaseries_,pearsoncorrtemp[:,0],color=Ecolor[i],linewidth=1.5,alpha=1,label=r'$E$ population')
ax[0].plot(symetaseries_,pearsoncorrtemp[:,1],color=Icolor[i],linewidth=1.5,alpha=1,label=r'$I$ population')
ax[0].scatter(symetaseries_,pearsoncorrtemp[:,0],s=20.0,c=Ecolor[i],marker='*',alpha=1)
ax[0].scatter(symetaseries_,pearsoncorrtemp[:,1],s=20.0,c=Icolor[i],marker='*',alpha=1)
ax[0].set_xlabel(r'$\eta$',font_format)
ax[0].set_ylabel(r'$\rho$[pearson]',font_format)
ax[0].spines[delstr].set_color('none')
ax[0].spines['right'].set_color('none')


ax[1].plot(symetaseries_,sigeigv*eigtemp/eigvAm[0],color='black',linewidth=1.5,alpha=1)
ax[1].scatter(symetaseries_,sigeigv*eigtemp/eigvAm[0],s=20.0,c='k',marker='*',alpha=1)
ax[1].set_xlabel(r'$\eta$',font_format)
ax[1].set_ylabel(r'$\lambda/\vert\lambda^O\vert$',font_format)
ax[1].spines['bottom'].set_color('none')
ax[1].spines['right'].set_color('none')

for i in range(1):
    ax[i].set_xlim(xlims)
    ax[i].set_ylim(ylims)
    ax[i].set_xticks([])#xticks)
    ax[i].set_yticks([])#yticks)
    ax[i].set_aspect('equal')
    # ax[i].legend([r'$E$ population',r'$I$ population'],prop={'family' : 'calibre', 'size'   : 12})
for i in range(1,2,1):
    ax[i].set_xlim(xlims)
    ax[i].set_ylim([-1.5,-1.0])
    ax[i].set_xticks([])#xticks)
    ax[i].set_yticks([])#yticks)
    ax[i].set_aspect('equal')
    # ax[i].legend(font_format)
